# ICCS261 Project: Creating a Player Efficiency Rating for League of Legends Esports

In [287]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import scipy.stats as stats

## Loading the dataset

In [ ]:
# Load the dataset with specified dtype
data = pd.read_csv('OraclesElixirLoLData2024.csv', low_memory=False)
data = data.drop(columns=['url'])
# Display the first few rows of the dataframe
data.head()

pd.reset_option('display.max_columns')

In [334]:
data = data[data['datacompleteness'] != 'partial']
data.reset_index(drop=True, inplace=True)
data.head()

,gameid,datacompleteness,league,year,split,playoffs,date,game,patch,participantid,...,opp_csat25,golddiffat25,xpdiffat25,csdiffat25,killsat25,assistsat25,deathsat25,opp_killsat25,opp_assistsat25,opp_deathsat25
0,LOLTMNT99_132542,complete,TSC,2024,Winter,0,2024-01-05 14:08:39,1,14.01,1,...,207.0,1928.0,1127.0,3.0,4.0,4.0,1.0,1.0,1.0,3.0
1,LOLTMNT99_132542,complete,TSC,2024,Winter,0,2024-01-05 14:08:39,1,14.01,2,...,154.0,2943.0,1955.0,0.0,5.0,11.0,1.0,2.0,3.0,2.0
2,LOLTMNT99_132542,complete,TSC,2024,Winter,0,2024-01-05 14:08:39,1,14.01,3,...,190.0,660.0,2223.0,18.0,1.0,8.0,3.0,1.0,4.0,3.0
3,LOLTMNT99_132542,complete,TSC,2024,Winter,0,2024-01-05 14:08:39,1,14.01,4,...,170.0,5016.0,2473.0,64.0,8.0,7.0,1.0,1.0,4.0,5.0
4,LOLTMNT99_132542,complete,TSC,2024,Winter,0,2024-01-05 14:08:39,1,14.01,5,...,31.0,2194.0,3049.0,-6.0,2.0,17.0,1.0,2.0,2.0,7.0


In [290]:
unique_leagues = data['league'].unique()
print(unique_leagues)

['TSC' 'LEC' 'LCKC' 'LCO' 'UL' 'LLA' 'LCK' 'PRM' 'LVP SL' 'LFL' 'PCS'
 'LJL' 'VCS' 'CBLOL' 'LCS' 'NACL' 'HM' 'GLL' 'ESLOL' 'NLC' 'TCL' 'LIT'
 'LPLOL' 'EBL' 'CBLOLA' 'AL' 'HC' 'LFL2' 'NEXO' 'LRN' 'LRS' 'EM' 'LAS'
 'CT' 'EPL' 'EWC' 'WLDs' 'CDF' 'PRMP' 'USP' 'HW' 'NLC Aurora Open' 'EBLPA'
 'GLLPA' 'IC']


In [291]:
tier1_leagues = ['LCS', 'LEC', 'LCK', 'LPL']
tier1_data = data[data['league'].isin(tier1_leagues)]
tier1_data.reset_index(drop=True, inplace=True)
tier1_data.head()

,gameid,datacompleteness,league,year,split,playoffs,date,game,patch,participantid,...,opp_csat25,golddiffat25,xpdiffat25,csdiffat25,killsat25,assistsat25,deathsat25,opp_killsat25,opp_assistsat25,opp_deathsat25
0,LOLTMNT06_13630,complete,LEC,2024,Winter,0,2024-01-13 16:10:20,1,14.01,1,...,230.0,940.0,-1286.0,-9.0,3.0,2.0,2.0,2.0,1.0,3.0
1,LOLTMNT06_13630,complete,LEC,2024,Winter,0,2024-01-13 16:10:20,1,14.01,2,...,151.0,320.0,1483.0,39.0,2.0,2.0,0.0,2.0,4.0,1.0
2,LOLTMNT06_13630,complete,LEC,2024,Winter,0,2024-01-13 16:10:20,1,14.01,3,...,256.0,-453.0,-3524.0,-54.0,2.0,1.0,1.0,0.0,3.0,0.0
3,LOLTMNT06_13630,complete,LEC,2024,Winter,0,2024-01-13 16:10:20,1,14.01,4,...,223.0,55.0,1091.0,32.0,2.0,2.0,2.0,3.0,1.0,2.0
4,LOLTMNT06_13630,complete,LEC,2024,Winter,0,2024-01-13 16:10:20,1,14.01,5,...,39.0,325.0,-316.0,-3.0,0.0,3.0,2.0,0.0,3.0,3.0


In [292]:
unique_players = tier1_data['playername'].unique().tolist()
len(unique_players)

183

In [293]:
# Select relevant columns for individual player statistics
player_stats_columns = [
    'playerid', 'playername', 'kills', 'deaths', 'assists', 'gamelength', 'goldat10', 'goldat15', 'goldat20', 'goldat25',
    'xpat10', 'xpat15', 'xpat20', 'xpat25', 'csat10', 'csat15', 'csat20', 'csat25', 'golddiffat10', 'golddiffat15', 
    'golddiffat20', 'golddiffat25', 'xpdiffat10', 'xpdiffat15', 'xpdiffat20', 'xpdiffat25', 'csdiffat10', 'csdiffat15', 
    'csdiffat20', 'csdiffat25'
]

# Filter and copy the data
player_stats_df = tier1_data[player_stats_columns].copy()

# Check for duplicate columns in the original data
print("Original DataFrame columns:", player_stats_df.columns.unique())

# Group by 'playerid' and aggregate statistics
player_stats_agg = (
    player_stats_df
    .groupby('playerid', as_index=False)  # Ensure we don’t create a multi-level index
    .agg({
        'playername': 'first',  # Keep the first name
        'kills': 'sum',
        'deaths': 'sum',
        'assists': 'sum',
        'gamelength': 'sum',
        'goldat10': 'mean',
        'goldat15': 'mean',
        'goldat20': 'mean',
        'goldat25': 'mean',
        'xpat10': 'mean',
        'xpat15': 'mean',
        'xpat20': 'mean',
        'xpat25': 'mean',
        'csat10': 'mean',
        'csat15': 'mean',
        'csat20': 'mean',
        'csat25': 'mean',
        'golddiffat10': 'mean',
        'golddiffat15': 'mean',
        'golddiffat20': 'mean',
        'golddiffat25': 'mean',
        'xpdiffat10': 'mean',
        'xpdiffat15': 'mean',
        'xpdiffat20': 'mean',
        'xpdiffat25': 'mean',
        'csdiffat10': 'mean',
        'csdiffat15': 'mean',
        'csdiffat20': 'mean',
        'csdiffat25': 'mean',
    })
)

# Recalculate KDA and GPM
player_stats_agg['KDA'] = (player_stats_agg['kills'] + player_stats_agg['assists']) / player_stats_agg['deaths'].replace(0, np.nan)
player_stats_agg['GPM'] = (
    player_stats_agg[['goldat10', 'goldat15', 'goldat20', 'goldat25']].mean(axis=1)
    / player_stats_agg['gamelength'].replace(0, np.nan)
)

# Replace NaN values (if needed)
player_stats_agg['KDA'] = player_stats_agg['KDA'].fillna(0)
player_stats_agg['GPM'] = player_stats_agg['GPM'].fillna(0)

# Verify the result
print(f"Number of unique players: {player_stats_agg.shape[0]}")  # Should be 183
print(f"Columns: {player_stats_agg.columns}")

# Display the aggregated dataframe
player_stats_agg


Original DataFrame columns: Index(['playerid', 'playername', 'kills', 'deaths', 'assists', 'gamelength',
       'goldat10', 'goldat15', 'goldat20', 'goldat25', 'xpat10', 'xpat15',
       'xpat20', 'xpat25', 'csat10', 'csat15', 'csat20', 'csat25',
       'golddiffat10', 'golddiffat15', 'golddiffat20', 'golddiffat25',
       'xpdiffat10', 'xpdiffat15', 'xpdiffat20', 'xpdiffat25', 'csdiffat10',
       'csdiffat15', 'csdiffat20', 'csdiffat25'],
      dtype='object')
Number of unique players: 182
Columns: Index(['playerid', 'playername', 'kills', 'deaths', 'assists', 'gamelength',
       'goldat10', 'goldat15', 'goldat20', 'goldat25', 'xpat10', 'xpat15',
       'xpat20', 'xpat25', 'csat10', 'csat15', 'csat20', 'csat25',
       'golddiffat10', 'golddiffat15', 'golddiffat20', 'golddiffat25',
       'xpdiffat10', 'xpdiffat15', 'xpdiffat20', 'xpdiffat25', 'csdiffat10',
       'csdiffat15', 'csdiffat20', 'csdiffat25', 'KDA', 'GPM'],
      dtype='object')


,playerid,playername,kills,deaths,assists,gamelength,goldat10,goldat15,goldat20,goldat25,...,xpdiffat10,xpdiffat15,xpdiffat20,xpdiffat25,csdiffat10,csdiffat15,csdiffat20,csdiffat25,KDA,GPM
0,oe:player:007be3acd5669d100711b13d79e2336,Sniper,165,210,249,112782,3213.816667,5034.683333,6941.916667,8875.431034,...,-135.200000,-249.200000,-434.033333,-602.465517,-4.216667,-6.850000,-9.450000,-10.241379,1.971429,0.053346
1,oe:player:010c1b0d37c2b7459b215dae7ffeeac,Cabochard,40,51,66,35438,3304.444444,5099.166667,6964.111111,8910.888889,...,1.611111,-113.055556,-450.055556,-539.111111,-0.611111,-4.388889,-10.500000,-12.888889,2.078431,0.171275
2,oe:player:027a5217b659e8bedb70978dea494d6,Irrelevant,143,130,251,99547,3353.000000,5426.134615,7489.711538,9488.274510,...,122.942308,326.519231,288.596154,121.372549,3.384615,7.269231,7.653846,8.960784,3.030769,0.064686
3,oe:player:035b9ae578574cdf728d8760483d804,Effort,3,46,68,26885,2522.928571,3779.000000,4957.285714,6173.928571,...,-143.214286,-722.714286,-1067.785714,-1472.500000,-14.928571,-25.071429,-30.214286,-34.642857,1.543478,0.162108
4,oe:player:03daae1844b9ad257324c3eedf41e28,Kiin,292,207,623,195780,3387.223301,5382.805825,7509.252427,9591.895833,...,148.097087,291.932039,468.747573,714.781250,5.436893,8.941748,10.174757,8.166667,4.420290,0.033036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,oe:player:fb6e61b8acb05df17b7e5ad4ae7c800,Chovy,467,171,597,195780,3595.398058,5931.097087,8403.893204,10919.666667,...,112.563107,295.582524,585.184466,1175.739583,6.106796,10.145631,18.242718,27.291667,6.222222,0.036840
178,oe:player:fbadcf84fcfb01a251de88055f631cb,Spica,49,78,229,57993,3318.800000,5085.000000,6844.800000,8509.035714,...,-22.433333,38.033333,-105.566667,-101.785714,0.866667,2.200000,2.500000,3.071429,3.564103,0.102416
179,oe:player:fd22b656b23ca07a276181a4953ba2c,Photon,138,117,232,88247,3403.760870,5351.260870,7593.173913,9884.045455,...,85.239130,42.913043,232.152174,393.409091,4.434783,3.978261,6.500000,10.818182,3.162393,0.074315
180,oe:player:fddde6d256bee26713f64ad9b2a9394,Palafox,134,134,230,89891,3481.159091,5594.113636,7783.250000,9698.904762,...,-76.522727,-271.159091,-564.750000,-1128.547619,-1.454545,-4.431818,-8.818182,-14.476190,2.716418,0.073860


In [294]:
player_unique = player_stats_agg['playername'].unique()
print(player_unique)

['Sniper' 'Cabochard' 'Irrelevant' 'Effort' 'Kiin' 'Delight' 'Jun'
 'CoreJJ' 'Perkz' 'eXyu' 'Lucid' 'Zven' 'Supa' 'Zwyroo' 'UmTi' 'Sylvie'
 'Dove' 'SAKEN' 'Patrik' 'Thanatos' 'Jackies' 'Impact' 'FATE' 'vital'
 'PerfecT' 'Teddy' 'Odoamne' 'Th3Antonio' 'Bdd' 'kyeahoo' 'Pollu'
 'BuLLDoG' 'Paduck' 'DnDn' 'Vladi' 'Samver' 'Razork' 'Hylissang' 'Busio'
 'Juhan' 'Sponge' 'Sheo' 'Isles' 'Execute' 'Yike' 'Exakick' 'Berserker'
 'Bvoy' 'Karis' 'Mask' 'JNX' 'Meech' 'Tactical' 'SeTab' 'Peach' 'Dhokla'
 'ISMA' 'Pullbae' 'Caps' 'Szygenda' 'Finn' 'Peyz' 'Comp' 'Vetheo' 'FIESTA'
 'Andil' 'ShowMaker' 'Duro' 'GIDEON' 'Kingen' 'Tomo' 'Raptor' 'Guwon'
 'DDoiV' 'Zeyzal' 'Adam' 'Clozer' 'Markoon' 'DuDu' 'Tomio' 'Rich'
 'Flakked' 'FBI' 'VULCAN' 'Canna' 'Nisqy' 'River' 'Jensen' 'Bwipo'
 'Pleata' 'Castle' 'Jojopyun' 'BrokenBlade' 'Aiming' 'Leaper' 'Jiwoo'
 'Casting' 'Callme' 'Contractz' 'Elyoya' 'Trymbi' 'Ice' 'nuc' 'Quid'
 'huhi' 'WildTurtle' 'Canyon' 'Bull' 'Taeyoon' 'Viper' 'Noah' 'Closer'
 'Hena' 'Zeus' 'Oll

In [295]:
lck_players_teams = tier1_data[tier1_data['league'] == 'LCK'][['playername', 'teamname']].drop_duplicates().sort_values(by='teamname')
print(lck_players_teams)
lec_players_teams = tier1_data[tier1_data['league'] == 'LEC'][['playername', 'teamname']].drop_duplicates().sort_values(by='teamname')
print(lec_players_teams)
lcs_players_teams = tier1_data[tier1_data['league'] == 'LCS'][['playername', 'teamname']].drop_duplicates().sort_values(by='teamname')
print(lcs_players_teams)
lpl_players_teams = tier1_data[tier1_data['league'] == 'LPL'][['playername', 'teamname']].drop_duplicates().sort_values(by='teamname')
print(lpl_players_teams)

     playername   teamname
269       Clear  BNK FearX
7629       Duro  BNK FearX
6246     Raptor  BNK FearX
275         NaN  BNK FearX
273     Execute  BNK FearX
...         ...        ...
213       Keria         T1
211       Faker         T1
210        Oner         T1
209        Zeus         T1
212    Gumayusi         T1

[83 rows x 2 columns]
     playername       teamname
35          NaN         Fnatic
29    Oscarinin         Fnatic
30       Razork         Fnatic
31     Humanoid         Fnatic
32         Noah         Fnatic
...         ...            ...
38       Vetheo  Team Vitality
37       Daglas  Team Vitality
6049     Lyncas  Team Vitality
36       Photon  Team Vitality
46          NaN  Team Vitality

[72 rows x 2 columns]
      playername           teamname
9567        Tomo        100 Thieves
449       Sniper        100 Thieves
450        River        100 Thieves
451         Quid        100 Thieves
452        Meech        100 Thieves
453         Eyla        100 Thieves
455   

We can see that the LPL players have all been dropped, likely due to data incompleteness. Let's filter all rows in the uncleaned original dataset for only LPL player related entries

In [296]:
data_unclean = pd.read_csv('OraclesElixirLoLData2024.csv', low_memory=False)
data_unclean

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat25,golddiffat25,xpdiffat25,csdiffat25,killsat25,assistsat25,deathsat25,opp_killsat25,opp_assistsat25,opp_deathsat25
0,10660-10660_game_1,partial,https://lpl.qq.com/es/stats.shtml?bmid=10660,DCup,2023,NaN,0,2024-01-01 05:13:15,1,13.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10660-10660_game_1,partial,https://lpl.qq.com/es/stats.shtml?bmid=10660,DCup,2023,NaN,0,2024-01-01 05:13:15,1,13.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10660-10660_game_1,partial,https://lpl.qq.com/es/stats.shtml?bmid=10660,DCup,2023,NaN,0,2024-01-01 05:13:15,1,13.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10660-10660_game_1,partial,https://lpl.qq.com/es/stats.shtml?bmid=10660,DCup,2023,NaN,0,2024-01-01 05:13:15,1,13.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10660-10660_game_1,partial,https://lpl.qq.com/es/stats.shtml?bmid=10660,DCup,2023,NaN,0,2024-01-01 05:13:15,1,13.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114283,LOLTMNT02_181861,complete,NaN,HW,2024,NaN,0,2024-11-03 21:08:36,5,14.18,...,200.0,716.0,2609.0,14.0,1.0,6.0,0.0,1.0,2.0,3.0
114284,LOLTMNT02_181861,complete,NaN,HW,2024,NaN,0,2024-11-03 21:08:36,5,14.18,...,264.0,-445.0,1274.0,-28.0,1.0,5.0,0.0,1.0,2.0,0.0
114285,LOLTMNT02_181861,complete,NaN,HW,2024,NaN,0,2024-11-03 21:08:36,5,14.18,...,36.0,151.0,1043.0,8.0,1.0,5.0,2.0,0.0,3.0,1.0
114286,LOLTMNT02_181861,complete,NaN,HW,2024,NaN,0,2024-11-03 21:08:36,5,14.18,...,862.0,-2295.0,-7637.0,8.0,4.0,9.0,9.0,9.0,23.0,4.0


In [297]:
lpl_teams = data_unclean[data_unclean['league'] == 'LPL']['teamname'].unique()
print(lpl_teams)

['Bilibili Gaming' 'Top Esports' 'Royal Never Give Up' 'EDward Gaming'
 'Oh My God' 'Team WE' "Anyone's Legend" 'FunPlus Phoenix'
 'Ninjas in Pyjamas' 'LGD Gaming' 'Ultra Prime' 'Invictus Gaming'
 'Weibo Gaming' 'Rare Atom' 'LNG Esports' 'ThunderTalk Gaming' 'JD Gaming']


To see all the regional LPL tournaments, we can filter for all the leagues that a LPL team that did not make international events played in

In [298]:
rare_atom_leagues = data_unclean[data_unclean['teamname'] == 'Rare Atom']['league'].unique()
print(rare_atom_leagues)

['DCup' 'LPL']


In [299]:
rare_atom_data = data_unclean[data_unclean['league'].isin(rare_atom_leagues)]
rare_atom_datacompleteness = rare_atom_data['datacompleteness'].unique()
print(rare_atom_datacompleteness)

['partial']


In [300]:
# Filter data_unclean for DCup and LPL leagues
dcup_lpl_unclean = data_unclean[data_unclean['league'].isin(['DCup', 'LPL'])]

# Find rows in dcup_lpl_unclean that are not in data
missing_data = dcup_lpl_unclean[~dcup_lpl_unclean['gameid'].isin(data['gameid'])]

# Display the missing data
missing_data

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat25,golddiffat25,xpdiffat25,csdiffat25,killsat25,assistsat25,deathsat25,opp_killsat25,opp_assistsat25,opp_deathsat25
0,10660-10660_game_1,partial,https://lpl.qq.com/es/stats.shtml?bmid=10660,DCup,2023,NaN,0,2024-01-01 05:13:15,1,13.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10660-10660_game_1,partial,https://lpl.qq.com/es/stats.shtml?bmid=10660,DCup,2023,NaN,0,2024-01-01 05:13:15,1,13.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10660-10660_game_1,partial,https://lpl.qq.com/es/stats.shtml?bmid=10660,DCup,2023,NaN,0,2024-01-01 05:13:15,1,13.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10660-10660_game_1,partial,https://lpl.qq.com/es/stats.shtml?bmid=10660,DCup,2023,NaN,0,2024-01-01 05:13:15,1,13.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10660-10660_game_1,partial,https://lpl.qq.com/es/stats.shtml?bmid=10660,DCup,2023,NaN,0,2024-01-01 05:13:15,1,13.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99799,11362-11362_game_3,partial,https://lpl.qq.com/es/stats.shtml?bmid=11362,LPL,2024,Summer,1,2024-08-30 10:16:18,3,14.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99800,11362-11362_game_3,partial,https://lpl.qq.com/es/stats.shtml?bmid=11362,LPL,2024,Summer,1,2024-08-30 10:16:18,3,14.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99801,11362-11362_game_3,partial,https://lpl.qq.com/es/stats.shtml?bmid=11362,LPL,2024,Summer,1,2024-08-30 10:16:18,3,14.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99802,11362-11362_game_3,partial,https://lpl.qq.com/es/stats.shtml?bmid=11362,LPL,2024,Summer,1,2024-08-30 10:16:18,3,14.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


It seems that practically all player data is missing for LPL. This means we will not be able to include players from the LPL region in our calculations. This should not be an issue however, since the LPER rating we are developing should still be able to be extrapolated from the rest of the dataset, we will just be unable to rank the LPL players, which is unfortunate.

In [301]:
player_stats_agg

,playerid,playername,kills,deaths,assists,gamelength,goldat10,goldat15,goldat20,goldat25,...,xpdiffat10,xpdiffat15,xpdiffat20,xpdiffat25,csdiffat10,csdiffat15,csdiffat20,csdiffat25,KDA,GPM
0,oe:player:007be3acd5669d100711b13d79e2336,Sniper,165,210,249,112782,3213.816667,5034.683333,6941.916667,8875.431034,...,-135.200000,-249.200000,-434.033333,-602.465517,-4.216667,-6.850000,-9.450000,-10.241379,1.971429,0.053346
1,oe:player:010c1b0d37c2b7459b215dae7ffeeac,Cabochard,40,51,66,35438,3304.444444,5099.166667,6964.111111,8910.888889,...,1.611111,-113.055556,-450.055556,-539.111111,-0.611111,-4.388889,-10.500000,-12.888889,2.078431,0.171275
2,oe:player:027a5217b659e8bedb70978dea494d6,Irrelevant,143,130,251,99547,3353.000000,5426.134615,7489.711538,9488.274510,...,122.942308,326.519231,288.596154,121.372549,3.384615,7.269231,7.653846,8.960784,3.030769,0.064686
3,oe:player:035b9ae578574cdf728d8760483d804,Effort,3,46,68,26885,2522.928571,3779.000000,4957.285714,6173.928571,...,-143.214286,-722.714286,-1067.785714,-1472.500000,-14.928571,-25.071429,-30.214286,-34.642857,1.543478,0.162108
4,oe:player:03daae1844b9ad257324c3eedf41e28,Kiin,292,207,623,195780,3387.223301,5382.805825,7509.252427,9591.895833,...,148.097087,291.932039,468.747573,714.781250,5.436893,8.941748,10.174757,8.166667,4.420290,0.033036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,oe:player:fb6e61b8acb05df17b7e5ad4ae7c800,Chovy,467,171,597,195780,3595.398058,5931.097087,8403.893204,10919.666667,...,112.563107,295.582524,585.184466,1175.739583,6.106796,10.145631,18.242718,27.291667,6.222222,0.036840
178,oe:player:fbadcf84fcfb01a251de88055f631cb,Spica,49,78,229,57993,3318.800000,5085.000000,6844.800000,8509.035714,...,-22.433333,38.033333,-105.566667,-101.785714,0.866667,2.200000,2.500000,3.071429,3.564103,0.102416
179,oe:player:fd22b656b23ca07a276181a4953ba2c,Photon,138,117,232,88247,3403.760870,5351.260870,7593.173913,9884.045455,...,85.239130,42.913043,232.152174,393.409091,4.434783,3.978261,6.500000,10.818182,3.162393,0.074315
180,oe:player:fddde6d256bee26713f64ad9b2a9394,Palafox,134,134,230,89891,3481.159091,5594.113636,7783.250000,9698.904762,...,-76.522727,-271.159091,-564.750000,-1128.547619,-1.454545,-4.431818,-8.818182,-14.476190,2.716418,0.073860


In [302]:
# Define weights for KDA and GPM
alpha = 1
beta = 1

# Calculate LPER
player_stats_agg['LPER'] = alpha * player_stats_agg['KDA'] + beta * player_stats_agg['GPM']

# Display the updated dataframe
player_stats_agg

,playerid,playername,kills,deaths,assists,gamelength,goldat10,goldat15,goldat20,goldat25,...,xpdiffat15,xpdiffat20,xpdiffat25,csdiffat10,csdiffat15,csdiffat20,csdiffat25,KDA,GPM,LPER
0,oe:player:007be3acd5669d100711b13d79e2336,Sniper,165,210,249,112782,3213.816667,5034.683333,6941.916667,8875.431034,...,-249.200000,-434.033333,-602.465517,-4.216667,-6.850000,-9.450000,-10.241379,1.971429,0.053346,2.024775
1,oe:player:010c1b0d37c2b7459b215dae7ffeeac,Cabochard,40,51,66,35438,3304.444444,5099.166667,6964.111111,8910.888889,...,-113.055556,-450.055556,-539.111111,-0.611111,-4.388889,-10.500000,-12.888889,2.078431,0.171275,2.249707
2,oe:player:027a5217b659e8bedb70978dea494d6,Irrelevant,143,130,251,99547,3353.000000,5426.134615,7489.711538,9488.274510,...,326.519231,288.596154,121.372549,3.384615,7.269231,7.653846,8.960784,3.030769,0.064686,3.095455
3,oe:player:035b9ae578574cdf728d8760483d804,Effort,3,46,68,26885,2522.928571,3779.000000,4957.285714,6173.928571,...,-722.714286,-1067.785714,-1472.500000,-14.928571,-25.071429,-30.214286,-34.642857,1.543478,0.162108,1.705587
4,oe:player:03daae1844b9ad257324c3eedf41e28,Kiin,292,207,623,195780,3387.223301,5382.805825,7509.252427,9591.895833,...,291.932039,468.747573,714.781250,5.436893,8.941748,10.174757,8.166667,4.420290,0.033036,4.453326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,oe:player:fb6e61b8acb05df17b7e5ad4ae7c800,Chovy,467,171,597,195780,3595.398058,5931.097087,8403.893204,10919.666667,...,295.582524,585.184466,1175.739583,6.106796,10.145631,18.242718,27.291667,6.222222,0.036840,6.259062
178,oe:player:fbadcf84fcfb01a251de88055f631cb,Spica,49,78,229,57993,3318.800000,5085.000000,6844.800000,8509.035714,...,38.033333,-105.566667,-101.785714,0.866667,2.200000,2.500000,3.071429,3.564103,0.102416,3.666519
179,oe:player:fd22b656b23ca07a276181a4953ba2c,Photon,138,117,232,88247,3403.760870,5351.260870,7593.173913,9884.045455,...,42.913043,232.152174,393.409091,4.434783,3.978261,6.500000,10.818182,3.162393,0.074315,3.236708
180,oe:player:fddde6d256bee26713f64ad9b2a9394,Palafox,134,134,230,89891,3481.159091,5594.113636,7783.250000,9698.904762,...,-271.159091,-564.750000,-1128.547619,-1.454545,-4.431818,-8.818182,-14.476190,2.716418,0.073860,2.790278


In [303]:
# Select the relevant columns
player_stats_summary = player_stats_agg[['playerid', 'playername', 'KDA', 'GPM', 'LPER']]

# Display the new table
player_stats_summary

,playerid,playername,KDA,GPM,LPER
0,oe:player:007be3acd5669d100711b13d79e2336,Sniper,1.971429,0.053346,2.024775
1,oe:player:010c1b0d37c2b7459b215dae7ffeeac,Cabochard,2.078431,0.171275,2.249707
2,oe:player:027a5217b659e8bedb70978dea494d6,Irrelevant,3.030769,0.064686,3.095455
3,oe:player:035b9ae578574cdf728d8760483d804,Effort,1.543478,0.162108,1.705587
4,oe:player:03daae1844b9ad257324c3eedf41e28,Kiin,4.420290,0.033036,4.453326
...,...,...,...,...,...
177,oe:player:fb6e61b8acb05df17b7e5ad4ae7c800,Chovy,6.222222,0.036840,6.259062
178,oe:player:fbadcf84fcfb01a251de88055f631cb,Spica,3.564103,0.102416,3.666519
179,oe:player:fd22b656b23ca07a276181a4953ba2c,Photon,3.162393,0.074315,3.236708
180,oe:player:fddde6d256bee26713f64ad9b2a9394,Palafox,2.716418,0.073860,2.790278


In [304]:
player_stats_summary.sort_values(by='LPER', ascending=False)

,playerid,playername,KDA,GPM,LPER
72,oe:player:6427b0c5a7c3106e92a7c11257a08c6,Guwon,7.750000,1.764423,9.514423
61,oe:player:5bbc2b92348d930ebd500cb9ce71bb7,Peyz,8.041379,0.035421,8.076801
109,oe:player:9573154ad8bc9459b4c0d940315054f,Viper,6.809249,0.032106,6.841354
93,oe:player:85104684d1eb7f4c0ce393441a0b669,Aiming,6.431953,0.031559,6.463512
46,oe:player:460ae5a7af8d68f7831b70307210ab3,Berserker,6.287879,0.074372,6.362251
...,...,...,...,...,...
126,oe:player:b1e10c40bba8608f1acfd3acdea43c2,GuGer,1.594595,0.077658,1.672253
79,oe:player:6ebdf3a3b56c113137109c45c2abc8f,Tomio,1.125000,0.546134,1.671134
116,oe:player:9daf04bfd9c8815cc827414f59f45af,Rascal,1.554945,0.051085,1.606030
108,oe:player:952237fce714ed1eca8ded17523bee9,Taeyoon,0.700000,0.893554,1.593554


In [305]:
# Group by 'playername' and count the number of games
player_game_counts = tier1_data.groupby('playername').size().reset_index(name='game_count')

# Merge player_game_counts with player_stats_agg to add playerid
player_game_counts = player_game_counts.merge(player_stats_agg[['playerid', 'playername']], on='playername', how='left')

# Display the result
player_game_counts

,playername,game_count,playerid
0,APA,62,oe:player:ea9ed09584feecae0991089f087d2ce
1,Adam,76,oe:player:6a9abcfebed9c636ca90560b7f49855
2,Aiming,111,oe:player:85104684d1eb7f4c0ce393441a0b669
3,Alvaro,75,oe:player:f5bf9bd732b85e4dcab5121e4769ae4
4,Andil,76,oe:player:5ef70ecd799a70ebbda239cd9a9f1e7
...,...,...,...
177,eXyu,18,oe:player:08936c56ab5a6453c58974a619651c3
178,huhi,44,oe:player:9191e392e93a6ef011ef76b5b83ee6c
179,kyeahoo,49,oe:player:308e48bf56284c09a1a9aadf55bc80c
180,nuc,81,oe:player:8f862245fbdb4d065e46c83630b9495


In [306]:
# Merge player_game_counts with player_stats_summary on 'playerid'
LPER_df = player_game_counts.merge(player_stats_summary, on='playerid')

# Display the merged dataframe
LPER_df

,playername_x,game_count,playerid,playername_y,KDA,GPM,LPER
0,APA,62,oe:player:ea9ed09584feecae0991089f087d2ce,APA,4.124183,0.060348,4.184531
1,Adam,76,oe:player:6a9abcfebed9c636ca90560b7f49855,Adam,3.114428,0.041663,3.156091
2,Aiming,111,oe:player:85104684d1eb7f4c0ce393441a0b669,Aiming,6.431953,0.031559,6.463512
3,Alvaro,75,oe:player:f5bf9bd732b85e4dcab5121e4769ae4,Alvaro,2.826087,0.030481,2.856568
4,Andil,76,oe:player:5ef70ecd799a70ebbda239cd9a9f1e7,Andil,2.243816,0.028037,2.271854
...,...,...,...,...,...,...,...
177,eXyu,18,oe:player:08936c56ab5a6453c58974a619651c3,eXyu,2.792453,0.166801,2.959254
178,huhi,44,oe:player:9191e392e93a6ef011ef76b5b83ee6c,huhi,2.484848,0.049807,2.534656
179,kyeahoo,49,oe:player:308e48bf56284c09a1a9aadf55bc80c,kyeahoo,2.881818,0.070338,2.952156
180,nuc,81,oe:player:8f862245fbdb4d065e46c83630b9495,nuc,3.563107,0.041683,3.604790


For statistical rigor, select only players with more than 30 games to be eligible in the LPER calculations

In [307]:
# Filter players with more than 30 games
LPER_sig = LPER_df[LPER_df['game_count'] > 30]

# Display the filtered dataframe
LPER_sig

,playername_x,game_count,playerid,playername_y,KDA,GPM,LPER
0,APA,62,oe:player:ea9ed09584feecae0991089f087d2ce,APA,4.124183,0.060348,4.184531
1,Adam,76,oe:player:6a9abcfebed9c636ca90560b7f49855,Adam,3.114428,0.041663,3.156091
2,Aiming,111,oe:player:85104684d1eb7f4c0ce393441a0b669,Aiming,6.431953,0.031559,6.463512
3,Alvaro,75,oe:player:f5bf9bd732b85e4dcab5121e4769ae4,Alvaro,2.826087,0.030481,2.856568
4,Andil,76,oe:player:5ef70ecd799a70ebbda239cd9a9f1e7,Andil,2.243816,0.028037,2.271854
...,...,...,...,...,...,...,...
172,Zeus,114,oe:player:98dd15c330fb5abb4508347265a7f76,Zeus,2.949324,0.029864,2.979188
176,Zwyroo,31,oe:player:1806cb4175771800d1d1ace2817103c,Zwyroo,3.923077,0.109507,4.032584
178,huhi,44,oe:player:9191e392e93a6ef011ef76b5b83ee6c,huhi,2.484848,0.049807,2.534656
179,kyeahoo,49,oe:player:308e48bf56284c09a1a9aadf55bc80c,kyeahoo,2.881818,0.070338,2.952156


In [308]:
LPER_sig.sort_values(by='LPER', ascending=False)

,playername_x,game_count,playerid,playername_y,KDA,GPM,LPER
124,Peyz,103,oe:player:5bbc2b92348d930ebd500cb9ce71bb7,Peyz,8.041379,0.035421,8.076801
163,Viper,112,oe:player:9573154ad8bc9459b4c0d940315054f,Viper,6.809249,0.032106,6.841354
2,Aiming,111,oe:player:85104684d1eb7f4c0ce393441a0b669,Aiming,6.431953,0.031559,6.463512
7,Berserker,47,oe:player:460ae5a7af8d68f7831b70307210ab3,Berserker,6.287879,0.074372,6.362251
27,Chovy,103,oe:player:fb6e61b8acb05df17b7e5ad4ae7c800,Chovy,6.222222,0.036840,6.259062
...,...,...,...,...,...,...,...
108,Morgan,82,oe:player:f73dde2d8c7879ae8d1fcdc03c4ffeb,Morgan,1.902041,0.039918,1.941959
127,Pollu,68,oe:player:3126bca5d4d2555ba3b460699508e16,Pollu,1.885057,0.031631,1.916689
146,Sponge,83,oe:player:3f4027dc08a489bd4d5390913f5dc71,Sponge,1.857651,0.035855,1.893506
40,DnDn,55,oe:player:341e10748bb6d388250fd1f013e45a4,DnDn,1.679739,0.057969,1.737707


Using eye test/trusted analyst ratings of players, it is clear that our first draft of the LPER significantly overvalues ADC players due to only using KDA and GPM. Let us try to adjust this.

In [309]:
# Add vision score to the player_stats_columns list
player_stats_columns.append('visionscore')

# Filter and copy the data including vision score
player_stats_df = tier1_data[player_stats_columns].copy()

# Group by 'playerid' and aggregate statistics including vision score
player_stats_agg = (
    player_stats_df
    .groupby('playerid', as_index=False)
    .agg({
        'playername': 'first',
        'kills': 'sum',
        'deaths': 'sum',
        'assists': 'sum',
        'gamelength': 'sum',
        'goldat10': 'mean',
        'goldat15': 'mean',
        'goldat20': 'mean',
        'goldat25': 'mean',
        'xpat10': 'mean',
        'xpat15': 'mean',
        'xpat20': 'mean',
        'xpat25': 'mean',
        'csat10': 'mean',
        'csat15': 'mean',
        'csat20': 'mean',
        'csat25': 'mean',
        'golddiffat10': 'mean',
        'golddiffat15': 'mean',
        'golddiffat20': 'mean',
        'golddiffat25': 'mean',
        'xpdiffat10': 'mean',
        'xpdiffat15': 'mean',
        'xpdiffat20': 'mean',
        'xpdiffat25': 'mean',
        'csdiffat10': 'mean',
        'csdiffat15': 'mean',
        'csdiffat20': 'mean',
        'csdiffat25': 'mean',
        'visionscore': 'mean'
    })
)

# Recalculate KDA, GPM, and include vision score in LPER
player_stats_agg['KDA'] = (player_stats_agg['kills'] + player_stats_agg['assists']) / player_stats_agg['deaths'].replace(0, np.nan)
player_stats_agg['GPM'] = (
    player_stats_agg[['goldat10', 'goldat15', 'goldat20', 'goldat25']].mean(axis=1)
    / player_stats_agg['gamelength'].replace(0, np.nan)
)

# Replace NaN values (if needed)
player_stats_agg['KDA'] = player_stats_agg['KDA'].fillna(0)
player_stats_agg['GPM'] = player_stats_agg['GPM'].fillna(0)

# Define weights for KDA, GPM, and vision score
alpha = 1
beta = 1
gamma = 0.1

# Calculate LPER including vision score
player_stats_agg['LPER'] = alpha * player_stats_agg['KDA'] + beta * player_stats_agg['GPM'] + gamma * player_stats_agg['visionscore']

# Display the updated dataframe
player_stats_agg

,playerid,playername,kills,deaths,assists,gamelength,goldat10,goldat15,goldat20,goldat25,...,xpdiffat20,xpdiffat25,csdiffat10,csdiffat15,csdiffat20,csdiffat25,visionscore,KDA,GPM,LPER
0,oe:player:007be3acd5669d100711b13d79e2336,Sniper,165,210,249,112782,3213.816667,5034.683333,6941.916667,8875.431034,...,-434.033333,-602.465517,-4.216667,-6.850000,-9.450000,-10.241379,26.950000,1.971429,0.053346,4.719775
1,oe:player:010c1b0d37c2b7459b215dae7ffeeac,Cabochard,40,51,66,35438,3304.444444,5099.166667,6964.111111,8910.888889,...,-450.055556,-539.111111,-0.611111,-4.388889,-10.500000,-12.888889,33.055556,2.078431,0.171275,5.555262
2,oe:player:027a5217b659e8bedb70978dea494d6,Irrelevant,143,130,251,99547,3353.000000,5426.134615,7489.711538,9488.274510,...,288.596154,121.372549,3.384615,7.269231,7.653846,8.960784,35.730769,3.030769,0.064686,6.668532
3,oe:player:035b9ae578574cdf728d8760483d804,Effort,3,46,68,26885,2522.928571,3779.000000,4957.285714,6173.928571,...,-1067.785714,-1472.500000,-14.928571,-25.071429,-30.214286,-34.642857,133.571429,1.543478,0.162108,15.062730
4,oe:player:03daae1844b9ad257324c3eedf41e28,Kiin,292,207,623,195780,3387.223301,5382.805825,7509.252427,9591.895833,...,468.747573,714.781250,5.436893,8.941748,10.174757,8.166667,37.815534,4.420290,0.033036,8.234879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,oe:player:fb6e61b8acb05df17b7e5ad4ae7c800,Chovy,467,171,597,195780,3595.398058,5931.097087,8403.893204,10919.666667,...,585.184466,1175.739583,6.106796,10.145631,18.242718,27.291667,54.922330,6.222222,0.036840,11.751295
178,oe:player:fbadcf84fcfb01a251de88055f631cb,Spica,49,78,229,57993,3318.800000,5085.000000,6844.800000,8509.035714,...,-105.566667,-101.785714,0.866667,2.200000,2.500000,3.071429,51.133333,3.564103,0.102416,8.779852
179,oe:player:fd22b656b23ca07a276181a4953ba2c,Photon,138,117,232,88247,3403.760870,5351.260870,7593.173913,9884.045455,...,232.152174,393.409091,4.434783,3.978261,6.500000,10.818182,39.804348,3.162393,0.074315,7.217143
180,oe:player:fddde6d256bee26713f64ad9b2a9394,Palafox,134,134,230,89891,3481.159091,5594.113636,7783.250000,9698.904762,...,-564.750000,-1128.547619,-1.454545,-4.431818,-8.818182,-14.476190,42.659091,2.716418,0.073860,7.056187


In [310]:
# Merge player_stats_agg with player_game_counts on 'playerid'
player_stats_agg = player_stats_agg.merge(player_game_counts[['playerid', 'game_count']], on='playerid')

# Display the merged dataframe
player_stats_agg

,playerid,playername,kills,deaths,assists,gamelength,goldat10,goldat15,goldat20,goldat25,...,xpdiffat25,csdiffat10,csdiffat15,csdiffat20,csdiffat25,visionscore,KDA,GPM,LPER,game_count
0,oe:player:007be3acd5669d100711b13d79e2336,Sniper,165,210,249,112782,3213.816667,5034.683333,6941.916667,8875.431034,...,-602.465517,-4.216667,-6.850000,-9.450000,-10.241379,26.950000,1.971429,0.053346,4.719775,60
1,oe:player:010c1b0d37c2b7459b215dae7ffeeac,Cabochard,40,51,66,35438,3304.444444,5099.166667,6964.111111,8910.888889,...,-539.111111,-0.611111,-4.388889,-10.500000,-12.888889,33.055556,2.078431,0.171275,5.555262,18
2,oe:player:027a5217b659e8bedb70978dea494d6,Irrelevant,143,130,251,99547,3353.000000,5426.134615,7489.711538,9488.274510,...,121.372549,3.384615,7.269231,7.653846,8.960784,35.730769,3.030769,0.064686,6.668532,52
3,oe:player:035b9ae578574cdf728d8760483d804,Effort,3,46,68,26885,2522.928571,3779.000000,4957.285714,6173.928571,...,-1472.500000,-14.928571,-25.071429,-30.214286,-34.642857,133.571429,1.543478,0.162108,15.062730,14
4,oe:player:03daae1844b9ad257324c3eedf41e28,Kiin,292,207,623,195780,3387.223301,5382.805825,7509.252427,9591.895833,...,714.781250,5.436893,8.941748,10.174757,8.166667,37.815534,4.420290,0.033036,8.234879,103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,oe:player:fb6e61b8acb05df17b7e5ad4ae7c800,Chovy,467,171,597,195780,3595.398058,5931.097087,8403.893204,10919.666667,...,1175.739583,6.106796,10.145631,18.242718,27.291667,54.922330,6.222222,0.036840,11.751295,103
178,oe:player:fbadcf84fcfb01a251de88055f631cb,Spica,49,78,229,57993,3318.800000,5085.000000,6844.800000,8509.035714,...,-101.785714,0.866667,2.200000,2.500000,3.071429,51.133333,3.564103,0.102416,8.779852,30
179,oe:player:fd22b656b23ca07a276181a4953ba2c,Photon,138,117,232,88247,3403.760870,5351.260870,7593.173913,9884.045455,...,393.409091,4.434783,3.978261,6.500000,10.818182,39.804348,3.162393,0.074315,7.217143,46
180,oe:player:fddde6d256bee26713f64ad9b2a9394,Palafox,134,134,230,89891,3481.159091,5594.113636,7783.250000,9698.904762,...,-1128.547619,-1.454545,-4.431818,-8.818182,-14.476190,42.659091,2.716418,0.073860,7.056187,44


In [311]:
# Filter players with more than or equal to 30 games
player_stats_agg = player_stats_agg[player_stats_agg['game_count'] >= 30]

# Display the filtered dataframe
player_stats_agg

,playerid,playername,kills,deaths,assists,gamelength,goldat10,goldat15,goldat20,goldat25,...,xpdiffat25,csdiffat10,csdiffat15,csdiffat20,csdiffat25,visionscore,KDA,GPM,LPER,game_count
0,oe:player:007be3acd5669d100711b13d79e2336,Sniper,165,210,249,112782,3213.816667,5034.683333,6941.916667,8875.431034,...,-602.465517,-4.216667,-6.850000,-9.450000,-10.241379,26.950000,1.971429,0.053346,4.719775,60
2,oe:player:027a5217b659e8bedb70978dea494d6,Irrelevant,143,130,251,99547,3353.000000,5426.134615,7489.711538,9488.274510,...,121.372549,3.384615,7.269231,7.653846,8.960784,35.730769,3.030769,0.064686,6.668532,52
4,oe:player:03daae1844b9ad257324c3eedf41e28,Kiin,292,207,623,195780,3387.223301,5382.805825,7509.252427,9591.895833,...,714.781250,5.436893,8.941748,10.174757,8.166667,37.815534,4.420290,0.033036,8.234879,103
5,oe:player:051ca2e31ba25123961b3df95fdef4c,Delight,89,251,1121,214336,2374.080357,3732.562500,4980.205357,6327.777778,...,591.851852,0.553571,1.357143,0.821429,0.694444,110.053571,4.820717,0.020312,15.846387,112
6,oe:player:054e5f8aeb885db6713f4a3e7328efc,Jun,63,283,920,165031,2413.541176,3706.105882,4933.576471,6232.630952,...,279.011905,-3.458824,-4.788235,-5.741176,-6.666667,121.600000,3.473498,0.026186,15.659684,85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,oe:player:fb6e61b8acb05df17b7e5ad4ae7c800,Chovy,467,171,597,195780,3595.398058,5931.097087,8403.893204,10919.666667,...,1175.739583,6.106796,10.145631,18.242718,27.291667,54.922330,6.222222,0.036840,11.751295,103
178,oe:player:fbadcf84fcfb01a251de88055f631cb,Spica,49,78,229,57993,3318.800000,5085.000000,6844.800000,8509.035714,...,-101.785714,0.866667,2.200000,2.500000,3.071429,51.133333,3.564103,0.102416,8.779852,30
179,oe:player:fd22b656b23ca07a276181a4953ba2c,Photon,138,117,232,88247,3403.760870,5351.260870,7593.173913,9884.045455,...,393.409091,4.434783,3.978261,6.500000,10.818182,39.804348,3.162393,0.074315,7.217143,46
180,oe:player:fddde6d256bee26713f64ad9b2a9394,Palafox,134,134,230,89891,3481.159091,5594.113636,7783.250000,9698.904762,...,-1128.547619,-1.454545,-4.431818,-8.818182,-14.476190,42.659091,2.716418,0.073860,7.056187,44


In [312]:
player_stats_agg.sort_values(by='LPER', ascending=False)

,playerid,playername,kills,deaths,assists,gamelength,goldat10,goldat15,goldat20,goldat25,...,xpdiffat25,csdiffat10,csdiffat15,csdiffat20,csdiffat25,visionscore,KDA,GPM,LPER,game_count
7,oe:player:05fafeaf23dc3285558a4e968970b9f,CoreJJ,58,142,656,119080,2433.951613,3777.322581,5087.000000,6417.700000,...,5.100000,-4.661290,-7.677419,-9.483871,-11.766667,117.451613,5.028169,0.037193,16.810524,62
129,oe:player:bb6ac5763c6576e104a7a2f59eb4ddf,Labrov,61,284,769,160981,2305.000000,3582.876543,4806.493827,6059.087500,...,-46.500000,-0.209877,-1.098765,-1.098765,-0.412500,129.209877,2.922535,0.026018,15.869541,81
5,oe:player:051ca2e31ba25123961b3df95fdef4c,Delight,89,251,1121,214336,2374.080357,3732.562500,4980.205357,6327.777778,...,591.851852,0.553571,1.357143,0.821429,0.694444,110.053571,4.820717,0.020312,15.846387,112
6,oe:player:054e5f8aeb885db6713f4a3e7328efc,Jun,63,283,920,165031,2413.541176,3706.105882,4933.576471,6232.630952,...,279.011905,-3.458824,-4.788235,-5.741176,-6.666667,121.600000,3.473498,0.026186,15.659684,85
115,oe:player:9d5052e9f9f4a63835f738441a3b1f6,Mikyx,72,369,930,170629,2433.272727,3764.113636,4990.443182,6301.178571,...,-196.916667,-3.170455,-5.318182,-6.670455,-8.952381,127.693182,2.715447,0.025624,15.510390,88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,oe:player:341e10748bb6d388250fd1f013e45a4,DnDn,86,153,171,106373,3294.527273,5126.309091,7082.527273,9161.943396,...,-720.566038,-1.927273,-3.690909,-8.327273,-9.339623,37.781818,1.679739,0.057969,5.515889,55
172,oe:player:f73dde2d8c7879ae8d1fcdc03c4ffeb,Morgan,169,245,297,155681,3288.426829,5191.060976,7244.597561,9133.787500,...,-785.362500,-1.585366,-2.756098,-1.439024,-4.225000,34.597561,1.902041,0.039918,5.401715,82
116,oe:player:9daf04bfd9c8815cc827414f59f45af,Rascal,103,182,180,120504,3206.919355,5147.419355,7175.451613,9094.175439,...,-778.736842,-5.919355,-9.612903,-11.064516,-13.105263,37.806452,1.554945,0.051085,5.386676,62
55,oe:player:5385f74da3e5deb5b66d9a1ee56cf97,Dhokla,115,154,181,89891,3205.000000,5078.636364,7079.636364,8890.547619,...,-778.000000,-3.454545,-3.590909,-3.318182,-5.404762,33.386364,1.922078,0.067453,5.328168,44


In [313]:
# Define weights for KDA, GPM, and vision score
alpha = 1
beta = 1
gamma = 0.025  # Halved the weight for vision score

# Calculate LPER including vision score
player_stats_agg['LPER'] = alpha * player_stats_agg['KDA'] + beta * player_stats_agg['GPM'] + gamma * player_stats_agg['visionscore']

# Display the updated dataframe
player_stats_agg.sort_values(by='LPER', ascending=False)

/var/folders/0h/ch124vqs7dj_96gjw7qq3ksh0000gn/T/ipykernel_14161/5607504.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_stats_agg['LPER'] = alpha * player_stats_agg['KDA'] + beta * player_stats_agg['GPM'] + gamma * player_stats_agg['visionscore']


,playerid,playername,kills,deaths,assists,gamelength,goldat10,goldat15,goldat20,goldat25,...,xpdiffat25,csdiffat10,csdiffat15,csdiffat20,csdiffat25,visionscore,KDA,GPM,LPER,game_count
61,oe:player:5bbc2b92348d930ebd500cb9ce71bb7,Peyz,519,145,647,195780,3515.349515,5783.844660,8064.728155,10375.239583,...,465.989583,-2.844660,-3.339806,-4.087379,-4.447917,51.932039,8.041379,0.035421,9.375102,103
109,oe:player:9573154ad8bc9459b4c0d940315054f,Viper,519,173,659,214336,3478.607143,5682.392857,7978.401786,10386.111111,...,420.361111,1.053571,4.125000,6.955357,9.138889,52.946429,6.809249,0.032106,8.165015,112
7,oe:player:05fafeaf23dc3285558a4e968970b9f,CoreJJ,58,142,656,119080,2433.951613,3777.322581,5087.000000,6417.700000,...,5.100000,-4.661290,-7.677419,-9.483871,-11.766667,117.451613,5.028169,0.037193,8.001653,62
93,oe:player:85104684d1eb7f4c0ce393441a0b669,Aiming,561,169,526,225016,3503.243243,5801.432432,8293.333333,10807.381818,...,983.190909,3.675676,8.306306,14.126126,20.836364,52.603604,6.431953,0.031559,7.778602,111
177,oe:player:fb6e61b8acb05df17b7e5ad4ae7c800,Chovy,467,171,597,195780,3595.398058,5931.097087,8403.893204,10919.666667,...,1175.739583,6.106796,10.145631,18.242718,27.291667,54.922330,6.222222,0.036840,7.632120,103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,oe:player:5385f74da3e5deb5b66d9a1ee56cf97,Dhokla,115,154,181,89891,3205.000000,5078.636364,7079.636364,8890.547619,...,-778.000000,-3.454545,-3.590909,-3.318182,-5.404762,33.386364,1.922078,0.067453,2.824190,44
172,oe:player:f73dde2d8c7879ae8d1fcdc03c4ffeb,Morgan,169,245,297,155681,3288.426829,5191.060976,7244.597561,9133.787500,...,-785.362500,-1.585366,-2.756098,-1.439024,-4.225000,34.597561,1.902041,0.039918,2.806898,82
0,oe:player:007be3acd5669d100711b13d79e2336,Sniper,165,210,249,112782,3213.816667,5034.683333,6941.916667,8875.431034,...,-602.465517,-4.216667,-6.850000,-9.450000,-10.241379,26.950000,1.971429,0.053346,2.698525,60
33,oe:player:341e10748bb6d388250fd1f013e45a4,DnDn,86,153,171,106373,3294.527273,5126.309091,7082.527273,9161.943396,...,-720.566038,-1.927273,-3.690909,-8.327273,-9.339623,37.781818,1.679739,0.057969,2.682253,55


The formula seems a little better, however we can see a player CoreJJ in the top three. This player is quite good but nowehere near top 3 by any analytical measure, and this is happening due to his dominance in his regional league LCS despite relatively poor showings in international tournaments. This highlights that to accurately calculate our LPER we must place more significance on more prestigious leages such as Worlds, MSI, and even inter-regional leagues, such as LCK > LCS. Let us try to find the weights for this.

LoLEsports.com, operated by Riot Games themselves, has a comprehensive Global Power Ranking, which includes a Regional Strength Score for each region across 2024. We will now add which region each player eligible in our LPER calculations is from, to adjust the weights for their performances.

In [314]:
# Define the team to region mapping
team_regions = {
    'Team BDS': 'LEC', 'G2 Esports': 'LEC', 'Rogue': 'LEC', 'SK Gaming': 'LEC', 'Karmine Corp': 'LEC', 
    'Fnatic': 'LEC', 'Team Vitality': 'LEC', 'GiantX': 'LEC', 'MAD Lions KOI': 'LEC', 'Team Heretics': 'LEC', 
    'DRX': 'LCK', 'Nongshim RedForce': 'LCK', 'Gen.G': 'LCK', 'T1': 'LCK', 'OKSavingsBank BRION': 'LCK', 
    'Dplus KIA': 'LCK', 'KT Rolster': 'LCK', 'BNK FearX': 'LCK', 'Hanwha Life Esports': 'LCK', 'Kwangdong Freecs': 'LCK', 
    'Team Liquid': 'LCS', '100 Thieves': 'LCS', 'Cloud9': 'LCS', 'NRG Kia': 'LCS', 'FlyQuest': 'LCS', 
    'Shopify Rebellion': 'LCS', 'Dignitas': 'LCS', 'Immortals': 'LCS'
}

# Add the Region column to player_stats_agg
player_stats_agg['Region'] = player_stats_agg['playerid'].map(lambda pid: team_regions.get(tier1_data[tier1_data['playerid'] == pid]['teamname'].values[0], 'Unknown'))

# Display the updated dataframe
player_stats_agg

/var/folders/0h/ch124vqs7dj_96gjw7qq3ksh0000gn/T/ipykernel_14161/524782237.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_stats_agg['Region'] = player_stats_agg['playerid'].map(lambda pid: team_regions.get(tier1_data[tier1_data['playerid'] == pid]['teamname'].values[0], 'Unknown'))


,playerid,playername,kills,deaths,assists,gamelength,goldat10,goldat15,goldat20,goldat25,...,csdiffat10,csdiffat15,csdiffat20,csdiffat25,visionscore,KDA,GPM,LPER,game_count,Region
0,oe:player:007be3acd5669d100711b13d79e2336,Sniper,165,210,249,112782,3213.816667,5034.683333,6941.916667,8875.431034,...,-4.216667,-6.850000,-9.450000,-10.241379,26.950000,1.971429,0.053346,2.698525,60,LCS
2,oe:player:027a5217b659e8bedb70978dea494d6,Irrelevant,143,130,251,99547,3353.000000,5426.134615,7489.711538,9488.274510,...,3.384615,7.269231,7.653846,8.960784,35.730769,3.030769,0.064686,3.988724,52,LEC
4,oe:player:03daae1844b9ad257324c3eedf41e28,Kiin,292,207,623,195780,3387.223301,5382.805825,7509.252427,9591.895833,...,5.436893,8.941748,10.174757,8.166667,37.815534,4.420290,0.033036,5.398714,103,LCK
5,oe:player:051ca2e31ba25123961b3df95fdef4c,Delight,89,251,1121,214336,2374.080357,3732.562500,4980.205357,6327.777778,...,0.553571,1.357143,0.821429,0.694444,110.053571,4.820717,0.020312,7.592369,112,LCK
6,oe:player:054e5f8aeb885db6713f4a3e7328efc,Jun,63,283,920,165031,2413.541176,3706.105882,4933.576471,6232.630952,...,-3.458824,-4.788235,-5.741176,-6.666667,121.600000,3.473498,0.026186,6.539684,85,LEC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,oe:player:fb6e61b8acb05df17b7e5ad4ae7c800,Chovy,467,171,597,195780,3595.398058,5931.097087,8403.893204,10919.666667,...,6.106796,10.145631,18.242718,27.291667,54.922330,6.222222,0.036840,7.632120,103,LCK
178,oe:player:fbadcf84fcfb01a251de88055f631cb,Spica,49,78,229,57993,3318.800000,5085.000000,6844.800000,8509.035714,...,0.866667,2.200000,2.500000,3.071429,51.133333,3.564103,0.102416,4.944852,30,LCS
179,oe:player:fd22b656b23ca07a276181a4953ba2c,Photon,138,117,232,88247,3403.760870,5351.260870,7593.173913,9884.045455,...,4.434783,3.978261,6.500000,10.818182,39.804348,3.162393,0.074315,4.231817,46,LEC
180,oe:player:fddde6d256bee26713f64ad9b2a9394,Palafox,134,134,230,89891,3481.159091,5594.113636,7783.250000,9698.904762,...,-1.454545,-4.431818,-8.818182,-14.476190,42.659091,2.716418,0.073860,3.856755,44,LCS


In [315]:
regional_strength_score = {
    'LCK': 1873, 
    'LPL': 1826, 
    'LEC': 1542,
    'LCS': 1486
}
regional_strength_score

{'LCK': 1873, 'LPL': 1826, 'LEC': 1542, 'LCS': 1486}

In [316]:
# Convert the regional_strength_score dictionary into a DataFrame
regional_strength_df = pd.DataFrame(list(regional_strength_score.items()), columns=['Region', 'RSS'])

# Display the DataFrame
print(regional_strength_df)

  Region   RSS
0    LCK  1873
1    LPL  1826
2    LEC  1542
3    LCS  1486


In [317]:
msi_row = data_unclean[data_unclean['league'] == 'MSI'].iloc[0]
print(msi_row)

gameid                                        11038-11038_game_1
datacompleteness                                         partial
url                 https://lpl.qq.com/es/stats.shtml?bmid=11038
league                                                       MSI
year                                                        2024
                                        ...                     
assistsat25                                                  NaN
deathsat25                                                   NaN
opp_killsat25                                                NaN
opp_assistsat25                                              NaN
opp_deathsat25                                               NaN
Name: 47712, Length: 161, dtype: object


MSI has partial datacompleteness, making it also unusable for LPER calculations

Since Worlds is the pinaccle of LoL esports, with the highest quality games between the best teams in each region, it makes sense to create a separate region strength score for Worlds, which we will set to 2000.

In [318]:
regional_strength_df.loc[len(regional_strength_df)] = ['WLDs', 2000]
regional_strength_df

,Region,RSS
0,LCK,1873
1,LPL,1826
2,LEC,1542
3,LCS,1486
4,WLDs,2000


Adding weights to tournaments means we must now calculate the weight of each match our players play. Therefore we must store every match a player has played.

In [319]:
data

,gameid,datacompleteness,league,year,split,playoffs,date,game,patch,participantid,...,opp_csat25,golddiffat25,xpdiffat25,csdiffat25,killsat25,assistsat25,deathsat25,opp_killsat25,opp_assistsat25,opp_deathsat25
0,LOLTMNT99_132542,complete,TSC,2024,Winter,0,2024-01-05 14:08:39,1,14.01,1,...,207.0,1928.0,1127.0,3.0,4.0,4.0,1.0,1.0,1.0,3.0
1,LOLTMNT99_132542,complete,TSC,2024,Winter,0,2024-01-05 14:08:39,1,14.01,2,...,154.0,2943.0,1955.0,0.0,5.0,11.0,1.0,2.0,3.0,2.0
2,LOLTMNT99_132542,complete,TSC,2024,Winter,0,2024-01-05 14:08:39,1,14.01,3,...,190.0,660.0,2223.0,18.0,1.0,8.0,3.0,1.0,4.0,3.0
3,LOLTMNT99_132542,complete,TSC,2024,Winter,0,2024-01-05 14:08:39,1,14.01,4,...,170.0,5016.0,2473.0,64.0,8.0,7.0,1.0,1.0,4.0,5.0
4,LOLTMNT99_132542,complete,TSC,2024,Winter,0,2024-01-05 14:08:39,1,14.01,5,...,31.0,2194.0,3049.0,-6.0,2.0,17.0,1.0,2.0,2.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97591,LOLTMNT02_181861,complete,HW,2024,NaN,0,2024-11-03 21:08:36,5,14.18,8,...,200.0,716.0,2609.0,14.0,1.0,6.0,0.0,1.0,2.0,3.0
97592,LOLTMNT02_181861,complete,HW,2024,NaN,0,2024-11-03 21:08:36,5,14.18,9,...,264.0,-445.0,1274.0,-28.0,1.0,5.0,0.0,1.0,2.0,0.0
97593,LOLTMNT02_181861,complete,HW,2024,NaN,0,2024-11-03 21:08:36,5,14.18,10,...,36.0,151.0,1043.0,8.0,1.0,5.0,2.0,0.0,3.0,1.0
97594,LOLTMNT02_181861,complete,HW,2024,NaN,0,2024-11-03 21:08:36,5,14.18,100,...,862.0,-2295.0,-7637.0,8.0,4.0,9.0,9.0,9.0,23.0,4.0


In [320]:
# Filter data to keep only rows where the league is in regional_strength_df['Region']
major_leagues_data = data[data['league'].isin(regional_strength_df['Region'])]

# Display the filtered dataframe
major_leagues_data

,gameid,datacompleteness,league,year,split,playoffs,date,game,patch,participantid,...,opp_csat25,golddiffat25,xpdiffat25,csdiffat25,killsat25,assistsat25,deathsat25,opp_killsat25,opp_assistsat25,opp_deathsat25
492,LOLTMNT06_13630,complete,LEC,2024,Winter,0,2024-01-13 16:10:20,1,14.01,1,...,230.0,940.0,-1286.0,-9.0,3.0,2.0,2.0,2.0,1.0,3.0
493,LOLTMNT06_13630,complete,LEC,2024,Winter,0,2024-01-13 16:10:20,1,14.01,2,...,151.0,320.0,1483.0,39.0,2.0,2.0,0.0,2.0,4.0,1.0
494,LOLTMNT06_13630,complete,LEC,2024,Winter,0,2024-01-13 16:10:20,1,14.01,3,...,256.0,-453.0,-3524.0,-54.0,2.0,1.0,1.0,0.0,3.0,0.0
495,LOLTMNT06_13630,complete,LEC,2024,Winter,0,2024-01-13 16:10:20,1,14.01,4,...,223.0,55.0,1091.0,32.0,2.0,2.0,2.0,3.0,1.0,2.0
496,LOLTMNT06_13630,complete,LEC,2024,Winter,0,2024-01-13 16:10:20,1,14.01,5,...,39.0,325.0,-316.0,-3.0,0.0,3.0,2.0,0.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97387,LOLTMNT05_90307,complete,WLDs,2024,NaN,0,2024-11-02 17:53:48,5,14.18,8,...,259.0,-1408.0,564.0,-14.0,1.0,0.0,1.0,3.0,0.0,0.0
97388,LOLTMNT05_90307,complete,WLDs,2024,NaN,0,2024-11-02 17:53:48,5,14.18,9,...,250.0,892.0,-49.0,35.0,1.0,1.0,0.0,0.0,0.0,0.0
97389,LOLTMNT05_90307,complete,WLDs,2024,NaN,0,2024-11-02 17:53:48,5,14.18,10,...,37.0,230.0,-290.0,-8.0,0.0,2.0,1.0,0.0,2.0,2.0
97390,LOLTMNT05_90307,complete,WLDs,2024,NaN,0,2024-11-02 17:53:48,5,14.18,100,...,967.0,-592.0,-846.0,-35.0,3.0,7.0,2.0,2.0,5.0,3.0


In [321]:
# Merge major_leagues_data with regional_strength_df on league and Region
major_leagues_data = major_leagues_data.merge(regional_strength_df, left_on='league', right_on='Region')
major_leagues_data = major_leagues_data.drop(columns='Region')
major_leagues_data

,gameid,datacompleteness,league,year,split,playoffs,date,game,patch,participantid,...,golddiffat25,xpdiffat25,csdiffat25,killsat25,assistsat25,deathsat25,opp_killsat25,opp_assistsat25,opp_deathsat25,RSS
0,LOLTMNT06_13630,complete,LEC,2024,Winter,0,2024-01-13 16:10:20,1,14.01,1,...,940.0,-1286.0,-9.0,3.0,2.0,2.0,2.0,1.0,3.0,1542
1,LOLTMNT06_13630,complete,LEC,2024,Winter,0,2024-01-13 16:10:20,1,14.01,2,...,320.0,1483.0,39.0,2.0,2.0,0.0,2.0,4.0,1.0,1542
2,LOLTMNT06_13630,complete,LEC,2024,Winter,0,2024-01-13 16:10:20,1,14.01,3,...,-453.0,-3524.0,-54.0,2.0,1.0,1.0,0.0,3.0,0.0,1542
3,LOLTMNT06_13630,complete,LEC,2024,Winter,0,2024-01-13 16:10:20,1,14.01,4,...,55.0,1091.0,32.0,2.0,2.0,2.0,3.0,1.0,2.0,1542
4,LOLTMNT06_13630,complete,LEC,2024,Winter,0,2024-01-13 16:10:20,1,14.01,5,...,325.0,-316.0,-3.0,0.0,3.0,2.0,0.0,3.0,3.0,1542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13039,LOLTMNT05_90307,complete,WLDs,2024,NaN,0,2024-11-02 17:53:48,5,14.18,8,...,-1408.0,564.0,-14.0,1.0,0.0,1.0,3.0,0.0,0.0,2000
13040,LOLTMNT05_90307,complete,WLDs,2024,NaN,0,2024-11-02 17:53:48,5,14.18,9,...,892.0,-49.0,35.0,1.0,1.0,0.0,0.0,0.0,0.0,2000
13041,LOLTMNT05_90307,complete,WLDs,2024,NaN,0,2024-11-02 17:53:48,5,14.18,10,...,230.0,-290.0,-8.0,0.0,2.0,1.0,0.0,2.0,2.0,2000
13042,LOLTMNT05_90307,complete,WLDs,2024,NaN,0,2024-11-02 17:53:48,5,14.18,100,...,-592.0,-846.0,-35.0,3.0,7.0,2.0,2.0,5.0,3.0,2000


In [322]:
major_leagues_data

,gameid,datacompleteness,league,year,split,playoffs,date,game,patch,participantid,...,golddiffat25,xpdiffat25,csdiffat25,killsat25,assistsat25,deathsat25,opp_killsat25,opp_assistsat25,opp_deathsat25,RSS
0,LOLTMNT06_13630,complete,LEC,2024,Winter,0,2024-01-13 16:10:20,1,14.01,1,...,940.0,-1286.0,-9.0,3.0,2.0,2.0,2.0,1.0,3.0,1542
1,LOLTMNT06_13630,complete,LEC,2024,Winter,0,2024-01-13 16:10:20,1,14.01,2,...,320.0,1483.0,39.0,2.0,2.0,0.0,2.0,4.0,1.0,1542
2,LOLTMNT06_13630,complete,LEC,2024,Winter,0,2024-01-13 16:10:20,1,14.01,3,...,-453.0,-3524.0,-54.0,2.0,1.0,1.0,0.0,3.0,0.0,1542
3,LOLTMNT06_13630,complete,LEC,2024,Winter,0,2024-01-13 16:10:20,1,14.01,4,...,55.0,1091.0,32.0,2.0,2.0,2.0,3.0,1.0,2.0,1542
4,LOLTMNT06_13630,complete,LEC,2024,Winter,0,2024-01-13 16:10:20,1,14.01,5,...,325.0,-316.0,-3.0,0.0,3.0,2.0,0.0,3.0,3.0,1542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13039,LOLTMNT05_90307,complete,WLDs,2024,NaN,0,2024-11-02 17:53:48,5,14.18,8,...,-1408.0,564.0,-14.0,1.0,0.0,1.0,3.0,0.0,0.0,2000
13040,LOLTMNT05_90307,complete,WLDs,2024,NaN,0,2024-11-02 17:53:48,5,14.18,9,...,892.0,-49.0,35.0,1.0,1.0,0.0,0.0,0.0,0.0,2000
13041,LOLTMNT05_90307,complete,WLDs,2024,NaN,0,2024-11-02 17:53:48,5,14.18,10,...,230.0,-290.0,-8.0,0.0,2.0,1.0,0.0,2.0,2.0,2000
13042,LOLTMNT05_90307,complete,WLDs,2024,NaN,0,2024-11-02 17:53:48,5,14.18,100,...,-592.0,-846.0,-35.0,3.0,7.0,2.0,2.0,5.0,3.0,2000


In [325]:
# Merge major_leagues_data with tier1_data to add visionscore
major_leagues_data = major_leagues_data.merge(
    tier1_data[['gameid', 'participantid', 'visionscore']], 
    on=['gameid', 'participantid'], 
    how='left'
)

major_leagues_data.rename(columns={'visionscore_y': 'visionscore'}, inplace=True)

# Display the updated dataframe
major_leagues_data

,gameid,datacompleteness,league,year,split,playoffs,date,game,patch,participantid,...,opp_killsat25,opp_assistsat25,opp_deathsat25,RSS,visionscore,KDA,GPM,KDA_normalized,GPM_normalized,visionscore
0,LOLTMNT06_13630,complete,LEC,2024,Winter,0,2024-01-13 16:10:20,1,14.01,1,...,2.0,1.0,3.0,1542,20.0,2.000000,18.298456,0.023810,0.183197,20.0
1,LOLTMNT06_13630,complete,LEC,2024,Winter,0,2024-01-13 16:10:20,1,14.01,2,...,2.0,4.0,1.0,1542,41.0,2.000000,13.967410,0.023810,0.139836,41.0
2,LOLTMNT06_13630,complete,LEC,2024,Winter,0,2024-01-13 16:10:20,1,14.01,3,...,0.0,3.0,0.0,1542,22.0,1.000000,14.655803,0.011905,0.146728,22.0
3,LOLTMNT06_13630,complete,LEC,2024,Winter,0,2024-01-13 16:10:20,1,14.01,4,...,3.0,1.0,2.0,1542,28.0,2.000000,15.959977,0.023810,0.159785,28.0
4,LOLTMNT06_13630,complete,LEC,2024,Winter,0,2024-01-13 16:10:20,1,14.01,5,...,0.0,3.0,3.0,1542,162.0,1.000000,10.128645,0.011905,0.101404,162.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13039,LOLTMNT05_90307,complete,WLDs,2024,NaN,0,2024-11-02 17:53:48,5,14.18,8,...,3.0,0.0,0.0,2000,NaN,10.000000,13.534920,0.119048,0.135506,NaN
13040,LOLTMNT05_90307,complete,WLDs,2024,NaN,0,2024-11-02 17:53:48,5,14.18,9,...,0.0,0.0,0.0,2000,NaN,6.000000,15.050181,0.071429,0.150676,NaN
13041,LOLTMNT05_90307,complete,WLDs,2024,NaN,0,2024-11-02 17:53:48,5,14.18,10,...,0.0,2.0,2.0,2000,NaN,1.500000,8.699948,0.017857,0.087100,NaN
13042,LOLTMNT05_90307,complete,WLDs,2024,NaN,0,2024-11-02 17:53:48,5,14.18,100,...,2.0,5.0,3.0,2000,NaN,1.916667,58.895499,0.022817,0.589638,NaN


In [329]:
# Step 1: Calculate KDA and GPM for each game
major_leagues_data['KDA'] = (
    (major_leagues_data['kills'] + major_leagues_data['assists']) / 
    major_leagues_data['deaths'].replace(0, np.nan)
)
major_leagues_data['GPM'] = (
    major_leagues_data[['goldat10', 'goldat15', 'goldat20', 'goldat25']].sum(axis=1) / 
    major_leagues_data['gamelength'].replace(0, np.nan)
)

# Step 2: Normalize KDA, GPM, and visionscore across all games
# Add a small constant to prevent division by zero in case of missing values
major_leagues_data['KDA_normalized'] = major_leagues_data['KDA'] / (major_leagues_data['KDA'].max())
major_leagues_data['GPM_normalized'] = major_leagues_data['GPM'] / (major_leagues_data['GPM'].max())
major_leagues_data['visionscore_normalized'] = major_leagues_data['visionscore'] / (major_leagues_data['visionscore'].max())

# Step 3: Calculate LPER for each game using normalized stats and RSS
major_leagues_data['LPER'] = (
    major_leagues_data['KDA_normalized'] + 
    major_leagues_data['GPM_normalized'] + 
    major_leagues_data['visionscore_normalized']
) * major_leagues_data['RSS']

# Step 4: Aggregate the LPER for each player
# Only consider valid rows (drop rows with missing player IDs or LPER values)
major_leagues_data = major_leagues_data.dropna(subset=['playerid', 'LPER'])

player_lper = major_leagues_data.groupby('playerid').agg({
    'KDA': 'mean',                # Average KDA across all games
    'GPM': 'mean',                # Average GPM across all games
    'LPER': 'mean'                # Average LPER across all games
}).reset_index()

# Display the player_lper dataframe
player_lper


ValueError: Cannot set a DataFrame with multiple columns to the single column visionscore_normalized

In [ ]:
# Merge player_lper with player_stats_agg to add playername
player_lper = player_lper.merge(data[['playerid', 'playername']], on='playerid', how='left')

# Display the updated dataframe
player_lper

,playerid,KDA,GPM,LPER,playername
0,oe:player:007be3acd5669d100711b13d79e2336,2.732779,12.777554,302.086817,Sniper
1,oe:player:007be3acd5669d100711b13d79e2336,2.732779,12.777554,302.086817,Sniper
2,oe:player:007be3acd5669d100711b13d79e2336,2.732779,12.777554,302.086817,Sniper
3,oe:player:007be3acd5669d100711b13d79e2336,2.732779,12.777554,302.086817,Sniper
4,oe:player:007be3acd5669d100711b13d79e2336,2.732779,12.777554,302.086817,Sniper
...,...,...,...,...,...
15237,oe:player:fe4329cec43193c425eb28a4a4b304a,4.196429,13.583972,380.737008,Upset
15238,oe:player:fe4329cec43193c425eb28a4a4b304a,4.196429,13.583972,380.737008,Upset
15239,oe:player:fe4329cec43193c425eb28a4a4b304a,4.196429,13.583972,380.737008,Upset
15240,oe:player:fe4329cec43193c425eb28a4a4b304a,4.196429,13.583972,380.737008,Upset


In [ ]:
unique_player_lper = player_lper.drop_duplicates()
unique_player_lper.reset_index(drop=True, inplace=True)
unique_player_lper

,playerid,KDA,GPM,LPER,playername
0,oe:player:007be3acd5669d100711b13d79e2336,2.732779,12.777554,246.893627,Sniper
1,oe:player:0101bf828e44b2630d500590c4ad461,2.625000,11.377573,290.315373,Gury
2,oe:player:010c1b0d37c2b7459b215dae7ffeeac,2.673529,12.570544,242.717006,Cabochard
3,oe:player:027a5217b659e8bedb70978dea494d6,4.389159,13.602561,288.989239,Irrelevant
4,oe:player:035b9ae578574cdf728d8760483d804,1.803846,9.207908,215.680759,Effort
...,...,...,...,...,...
228,oe:player:fbadcf84fcfb01a251de88055f631cb,4.521429,12.379240,262.848580,Spica
229,oe:player:fbdc4428b2d095c83aebeed5da4e4b2,5.721429,12.275227,381.474624,Xun
230,oe:player:fd22b656b23ca07a276181a4953ba2c,3.294246,13.694198,271.887817,Photon
231,oe:player:fddde6d256bee26713f64ad9b2a9394,3.057724,13.187942,247.496123,Palafox


In [ ]:
unique_players = unique_player_lper['playername'].unique()
print(unique_players)

['Sniper' 'Gury' 'Cabochard' 'Irrelevant' 'Effort' 'Kiin' 'Delight' 'Jun'
 'CoreJJ' 'Perkz' 'eXyu' '369' 'Breathe' 'Tian' 'Lucid' 'Zven' 'Supa'
 'dyNquedo' 'Levi' 'Vsta' 'Zwyroo' 'UmTi' 'Sylvie' 'Keine' 'TitaN' 'Dove'
 'SAKEN' 'Patrik' 'Thanatos' 'Bie' 'Jackies' 'Ceo' 'Elio' 'Impact' 'FATE'
 'vital' 'CarioK' 'PerfecT' 'Teddy' 'Odoamne' 'Th3Antonio' 'Kati' 'Bdd'
 'kyeahoo' 'Pollu' 'BuLLDoG' 'Woody' 'Paduck' 'Emo' 'DnDn' 'Vladi' 'Evi'
 'Samver' 'Razork' 'Light' 'Hylissang' 'Busio' 'Juhan' 'Sponge' 'Sheo'
 'Isles' 'Execute' 'Yike' 'Exakick' 'Berserker' 'Bvoy' 'Karis' 'Mask'
 'JNX' 'Meech' 'Tactical' 'DasheR' 'SeTab' 'Peach' 'Dhokla' 'Nanaue'
 'ISMA' 'Pullbae' 'Caps' 'Szygenda' 'Weiwei' 'Finn' 'Peyz' 'Comp' 'Vetheo'
 'FIESTA' 'Andil' 'ShowMaker' 'Duro' 'Crisp' 'GIDEON' 'Shogun' 'Kingen'
 'Tarzan' 'Tomo' 'Raptor' 'Guwon' 'Zika' 'DDoiV' 'Zeyzal' 'Adam' 'Clozer'
 'Markoon' 'DuDu' 'Creme' 'Tomio' 'Rich' 'Flakked' 'Scout' 'Azhi' 'FBI'
 'Kiaya' 'VULCAN' 'Canna' 'Nisqy' 'River' 'Jensen' 'Bwipo' '

In [ ]:
# Filter unique_player_lper to keep only players that are in player_stats_agg
filtered_unique_player_lper = unique_player_lper[unique_player_lper['playerid'].isin(player_stats_agg['playerid'])]

# Display the filtered dataframe
filtered_unique_player_lper.reset_index(drop=True, inplace=True)
filtered_unique_player_lper

,playerid,KDA,GPM,LPER,playername
0,oe:player:007be3acd5669d100711b13d79e2336,2.732779,12.777554,246.893627,Sniper
1,oe:player:027a5217b659e8bedb70978dea494d6,4.389159,13.602561,288.989239,Irrelevant
2,oe:player:03daae1844b9ad257324c3eedf41e28,4.929987,13.417252,355.037946,Kiin
3,oe:player:051ca2e31ba25123961b3df95fdef4c,5.809535,9.146655,300.184092,Delight
4,oe:player:054e5f8aeb885db6713f4a3e7328efc,4.815923,9.056134,230.932922,Jun
...,...,...,...,...,...
124,oe:player:fb6e61b8acb05df17b7e5ad4ae7c800,6.816667,14.972948,429.848817,Chovy
125,oe:player:fbadcf84fcfb01a251de88055f631cb,4.521429,12.379240,262.848580,Spica
126,oe:player:fd22b656b23ca07a276181a4953ba2c,3.294246,13.694198,271.887817,Photon
127,oe:player:fddde6d256bee26713f64ad9b2a9394,3.057724,13.187942,247.496123,Palafox


In [ ]:
final_lper = filtered_unique_player_lper[['playername', 'LPER']].sort_values(by='LPER', ascending=False)
final_lper

,playername,LPER
38,Peyz,438.442005
124,Chovy,429.848817
61,Aiming,401.850029
122,Zeka,397.974134
74,Viper,396.768543
...,...,...
27,Isles,199.986511
109,Targamas,194.855101
23,Hylissang,192.867883
71,huhi,190.777662


In [ ]:
unique_players = final_lper['playername'].unique()
print(unique_players)

['Peyz' 'Chovy' 'Aiming' 'Zeka' 'Viper' 'Bdd' 'Canyon' 'Gumayusi' 'Deft'
 'Faker' 'Peanut' 'Kiin' 'Leaper' 'Pyosik' 'ShowMaker' 'Oner' 'Clozer'
 'BuLLDoG' 'Noah' 'Zeus' 'Hena' 'Fisher' 'Quad' 'Doran' 'kyeahoo' 'Massu'
 'Hans Sama' 'Teddy' 'Jiwoo' 'Ice' 'Lucid' 'Raptor' 'Yeon' 'Humanoid'
 'PerfecT' 'Quid' 'Caps' 'Karis' 'Bull' 'Callme' 'DuDu' 'Supa' 'Yike'
 'Cuzz' 'Willer' 'Envyy' 'Zwyroo' 'nuc' 'APA' 'Berserker' 'Inspired'
 'YoungJae' 'Exakick' 'Tomo' 'Delight' 'Kingen' 'DnDn' 'Keria' 'Morgan'
 'Nisqy' 'Razork' 'Lehends' 'Sylvie' 'Clear' 'Elyoya' 'GIDEON' 'SeTab'
 'Sheo' 'Jojopyun' 'Vetheo' 'Flakked' 'UmTi' 'Fresskowy' 'Irrelevant'
 'Carzzy' 'BrokenBlade' 'Jensen' 'ISMA' 'Blaber' 'Jackies' 'Zven' 'Patrik'
 'Rascal' 'Impact' 'Meech' 'Oscarinin' 'BeryL' 'Upset' 'Adam' 'FBI'
 'Bwipo' 'Jankos' 'River' 'Photon' 'Insanity' 'Myrwn' 'Sponge' 'Daglas'
 'Kellin' 'Spica' 'Peter' 'Wunder' 'Doss' 'Duro' 'Palafox' 'Sniper'
 'Contractz' 'CoreJJ' 'Busio' 'Jun' 'Execute' 'VULCAN' 'Alvaro' 'Eyla'
 'Plea

In [ ]:
# Merge final_lper with player_game_counts to add game count
final_lper_with_game_count = final_lper.merge(player_game_counts[['playername', 'game_count']], left_on='playername', right_on='playername')

# Display the updated dataframe
final_lper_with_game_count.sort_values(by='game_count', ascending=False)

,playername,LPER,game_count
19,Zeus,347.224197,114
7,Gumayusi,384.375585,114
9,Faker,367.199569,114
57,Keria,298.470841,114
3,Zeka,397.974134,112
...,...,...,...
116,FakeGod,226.396317,30
99,Spica,262.848580,30
80,Zven,282.909980,30
94,Insanity,270.992347,30


In [ ]:
final_lper

,playername,LPER
38,Peyz,438.442005
124,Chovy,429.848817
61,Aiming,401.850029
122,Zeka,397.974134
74,Viper,396.768543
...,...,...
27,Isles,199.986511
109,Targamas,194.855101
23,Hylissang,192.867883
71,huhi,190.777662


The datasets and tables are becoming impossible to manage. Let us clean up. We know now that we must calculate the LPER based on a combination of KDA, KP, GPM, Vision Score, and Objective Control. Let's create a new table where we remove all other irrelevant columns.

In [337]:
lper_data = data.copy()
lper_data

columns = lper_data.columns.tolist()
print(columns)

['gameid', 'datacompleteness', 'league', 'year', 'split', 'playoffs', 'date', 'game', 'patch', 'participantid', 'side', 'position', 'playername', 'playerid', 'teamname', 'teamid', 'champion', 'ban1', 'ban2', 'ban3', 'ban4', 'ban5', 'pick1', 'pick2', 'pick3', 'pick4', 'pick5', 'gamelength', 'result', 'kills', 'deaths', 'assists', 'teamkills', 'teamdeaths', 'doublekills', 'triplekills', 'quadrakills', 'pentakills', 'firstblood', 'firstbloodkill', 'firstbloodassist', 'firstbloodvictim', 'team kpm', 'ckpm', 'firstdragon', 'dragons', 'opp_dragons', 'elementaldrakes', 'opp_elementaldrakes', 'infernals', 'mountains', 'clouds', 'oceans', 'chemtechs', 'hextechs', 'dragons (type unknown)', 'elders', 'opp_elders', 'firstherald', 'heralds', 'opp_heralds', 'void_grubs', 'opp_void_grubs', 'firstbaron', 'barons', 'opp_barons', 'firsttower', 'towers', 'opp_towers', 'firstmidtower', 'firsttothreetowers', 'turretplates', 'opp_turretplates', 'inhibitors', 'opp_inhibitors', 'damagetochampions', 'dpm', 'da

In [340]:
lper_data.drop(columns=['datacompleteness', 'year', 'patch', 'champion', 'ban1', 'ban2', 'ban3', 'ban4', 'ban5', 'pick1', 'pick2', 'pick3', 'pick4', 'pick5', 'infernals', 'mountains', 'clouds', 'oceans', 'chemtechs', 'hextechs', 'dragons (type unknown)'], errors='ignore', inplace=True)
lper_data

,gameid,league,split,playoffs,date,game,participantid,side,position,playername,...,opp_csat25,golddiffat25,xpdiffat25,csdiffat25,killsat25,assistsat25,deathsat25,opp_killsat25,opp_assistsat25,opp_deathsat25
0,LOLTMNT99_132542,TSC,Winter,0,2024-01-05 14:08:39,1,1,Blue,top,Taegyeong,...,207.0,1928.0,1127.0,3.0,4.0,4.0,1.0,1.0,1.0,3.0
1,LOLTMNT99_132542,TSC,Winter,0,2024-01-05 14:08:39,1,2,Blue,jng,CHEF,...,154.0,2943.0,1955.0,0.0,5.0,11.0,1.0,2.0,3.0,2.0
2,LOLTMNT99_132542,TSC,Winter,0,2024-01-05 14:08:39,1,3,Blue,mid,Nakyung,...,190.0,660.0,2223.0,18.0,1.0,8.0,3.0,1.0,4.0,3.0
3,LOLTMNT99_132542,TSC,Winter,0,2024-01-05 14:08:39,1,4,Blue,bot,Ruep,...,170.0,5016.0,2473.0,64.0,8.0,7.0,1.0,1.0,4.0,5.0
4,LOLTMNT99_132542,TSC,Winter,0,2024-01-05 14:08:39,1,5,Blue,sup,Carry,...,31.0,2194.0,3049.0,-6.0,2.0,17.0,1.0,2.0,2.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97591,LOLTMNT02_181861,HW,NaN,0,2024-11-03 21:08:36,5,8,Red,mid,Phantomles,...,200.0,716.0,2609.0,14.0,1.0,6.0,0.0,1.0,2.0,3.0
97592,LOLTMNT02_181861,HW,NaN,0,2024-11-03 21:08:36,5,9,Red,bot,raining,...,264.0,-445.0,1274.0,-28.0,1.0,5.0,0.0,1.0,2.0,0.0
97593,LOLTMNT02_181861,HW,NaN,0,2024-11-03 21:08:36,5,10,Red,sup,XERSUS,...,36.0,151.0,1043.0,8.0,1.0,5.0,2.0,0.0,3.0,1.0
97594,LOLTMNT02_181861,HW,NaN,0,2024-11-03 21:08:36,5,100,Blue,team,NaN,...,862.0,-2295.0,-7637.0,8.0,4.0,9.0,9.0,9.0,23.0,4.0


In [341]:
columns = lper_data.columns.tolist()
print(columns)

['gameid', 'league', 'split', 'playoffs', 'date', 'game', 'participantid', 'side', 'position', 'playername', 'playerid', 'teamname', 'teamid', 'gamelength', 'result', 'kills', 'deaths', 'assists', 'teamkills', 'teamdeaths', 'doublekills', 'triplekills', 'quadrakills', 'pentakills', 'firstblood', 'firstbloodkill', 'firstbloodassist', 'firstbloodvictim', 'team kpm', 'ckpm', 'firstdragon', 'dragons', 'opp_dragons', 'elementaldrakes', 'opp_elementaldrakes', 'elders', 'opp_elders', 'firstherald', 'heralds', 'opp_heralds', 'void_grubs', 'opp_void_grubs', 'firstbaron', 'barons', 'opp_barons', 'firsttower', 'towers', 'opp_towers', 'firstmidtower', 'firsttothreetowers', 'turretplates', 'opp_turretplates', 'inhibitors', 'opp_inhibitors', 'damagetochampions', 'dpm', 'damageshare', 'damagetakenperminute', 'damagemitigatedperminute', 'wardsplaced', 'wpm', 'wardskilled', 'wcpm', 'controlwardsbought', 'visionscore', 'vspm', 'totalgold', 'earnedgold', 'earned gpm', 'earnedgoldshare', 'goldspent', 'gsp

Let us focus on simply making our base LPER formula effective. Therefore let us focus on one league. We will choose LCK due to it usually being considered the most consistent region.

In [343]:
lck_data = lper_data[lper_data['league'] == 'LCK']
lck_data.reset_index(drop=True, inplace=True)
lck_data

,gameid,league,split,playoffs,date,game,participantid,side,position,playername,...,opp_csat25,golddiffat25,xpdiffat25,csdiffat25,killsat25,assistsat25,deathsat25,opp_killsat25,opp_assistsat25,opp_deathsat25
0,LOLTMNT02_22165,LCK,Spring,0,2024-01-17 08:20:00,1,1,Blue,top,Rascal,...,218.0,-231.0,-760.0,10.0,0.0,2.0,1.0,1.0,2.0,3.0
1,LOLTMNT02_22165,LCK,Spring,0,2024-01-17 08:20:00,1,2,Blue,jng,Sponge,...,147.0,963.0,-186.0,21.0,4.0,0.0,1.0,1.0,2.0,1.0
2,LOLTMNT02_22165,LCK,Spring,0,2024-01-17 08:20:00,1,3,Blue,mid,SeTab,...,235.0,-1244.0,-1329.0,-14.0,2.0,3.0,1.0,2.0,1.0,2.0
3,LOLTMNT02_22165,LCK,Spring,0,2024-01-17 08:20:00,1,4,Blue,bot,Teddy,...,230.0,1728.0,-272.0,7.0,2.0,3.0,2.0,1.0,1.0,1.0
4,LOLTMNT02_22165,LCK,Spring,0,2024-01-17 08:20:00,1,5,Blue,sup,Pleata,...,13.0,132.0,-212.0,14.0,0.0,5.0,0.0,0.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5779,LOLTMNT03_147405,LCK,Summer,1,2024-09-08 10:43:57,5,8,Red,mid,Zeka,...,253.0,157.0,-50.0,-4.0,2.0,4.0,0.0,0.0,1.0,1.0
5780,LOLTMNT03_147405,LCK,Summer,1,2024-09-08 10:43:57,5,9,Red,bot,Viper,...,255.0,974.0,227.0,-9.0,2.0,4.0,0.0,0.0,2.0,1.0
5781,LOLTMNT03_147405,LCK,Summer,1,2024-09-08 10:43:57,5,10,Red,sup,Delight,...,23.0,251.0,1218.0,12.0,0.0,7.0,1.0,0.0,3.0,3.0
5782,LOLTMNT03_147405,LCK,Summer,1,2024-09-08 10:43:57,5,100,Blue,team,NaN,...,896.0,-2944.0,-1974.0,-4.0,3.0,8.0,7.0,7.0,22.0,3.0


In [346]:
pd.set_option('display.max_columns', None)
lck_data

,gameid,league,split,playoffs,date,game,participantid,side,position,playername,playerid,teamname,teamid,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,elders,opp_elders,firstherald,heralds,opp_heralds,void_grubs,opp_void_grubs,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,turretplates,opp_turretplates,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,gpr,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15,goldat20,xpat20,csat20,opp_goldat20,opp_xpat20,opp_csat20,golddiffat20,xpdiffat20,csdiffat20,killsat20,assistsat20,deathsat20,opp_killsat20,opp_assistsat20,opp_deathsat20,goldat25,xpat25,csat25,opp_goldat25,opp_xpat25,opp_csat25,golddiffat25,xpdiffat25,csdiffat25,killsat25,assistsat25,deathsat25,opp_killsat25,opp_assistsat25,opp_deathsat25
0,LOLTMNT02_22165,LCK,Spring,0,2024-01-17 08:20:00,1,1,Blue,top,Rascal,oe:player:9daf04bfd9c8815cc827414f59f45af,DRX,oe:team:101f8589e58c724c1dcd5a9c1555277,2507,0,0,2,4,14,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3351,0.6701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,12881,308.2808,0.119309,879.7048,3157.8620,18,0.4308,13,0.3111,6,61,1.4599,14627,9237,221.0690,0.201840,14350,NaN,NaN,352.0,347.0,5,NaN,NaN,8.4244,3479.0,4733.0,90.0,3237.0,5052.0,86.0,242.0,-319.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,5366.0,7959.0,145.0,4998.0,7764.0,141.0,368.0,195.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,7243.0,10718.0,197.0,6940.0,10604.0,194.0,303.0,114.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,8872.0,12581.0,228.0,9103.0,13341.0,218.0,-231.0,-760.0,10.0,0.0,2.0,1.0,1.0,2.0,3.0
1,LOLTMNT02_22165,LCK,Spring,0,2024-01-17 08:20:00,1,2,Blue,jng,Sponge,oe:player:3f4027dc08a489bd4d5390913f5dc71,DRX,oe:team:101f8589e58c724c1dcd5a9c1555277,2507,0,6,4,3,14,14,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.3351,0.6701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,11256,269.3897,0.104258,889.7806,1548.7036,20,0.4787,20,0.4787,20,79,1.8907,13387,7997,191.3921,0.174745,12500,NaN,NaN,216.0,50.0,166,NaN,NaN,5.1695,3588.0,3651.0,73.0,2874.0,3253.0,58.0,714.0,398.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,5397.0,5531.0,107.0,4519.0,4849.0,76.0,878.0,682.0,31.0,2.0,0.0,0.0,1.0,0.0,1.0,7225.0,7939.0,140.0,6094.0,7462.0,117.0,1131.0,477.0,23.0,3.0,0.0,0.0,1.0,0.0,1.0,8897.0,10105.0,168.0,7934.0,10291.0,147.0,963.0,-186.0,21.0,4.0,0.0,1.0,1.0,2.0,1.0
2,LOLTMNT02_22165,LCK,Spring,0,2024-01-17 08:20:00,1,3,Blue,mid,SeTab,oe:player:4ff7b91bbc9492e9f96d28dda496653,DRX,oe:team:101f8589e58c724c1dcd5a9c1555277,2507,0,2,3,6,14,14,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.3351,0.6701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,25837,618.3566,0.239313,544.7866,452.6446,19,0.4547,10,0.2393,8,43,1.0291,15142,9752,233.3945,0.213093,14000,NaN,NaN,338.0,338.0,0,NaN,NaN,8.0893,3493.0,5088.0,93.0,3015.0,4682.0,75.0,478.0,406.0,18.0,0.0,1.0,0.0,0.0,0.0,1.0,5588.0,8030.0,150.0,4833.0,7571.0,128.0,755.0,459.0,22.0,0.0,1.0,0.0,0.0,0.0,1.0,8101.0,11383.0,206.0,6482.0,10069.0,171.0,1619.0,1314.0,35.0,1.0,1.0,0.0,0.0,1.0,2.0,9566.0,12887.0,221.0,10810.0,14216.0,235.0,-1244.0,-1329.0,-14.0,2.0,3.0,1.0,2.0,1.0,2.0
3,LOLTMNT02_22165,LCK,Spring,0,2024-01-17 08:20:00,1,4,Blue

In [ ]:
pd.reset_option('display.max_columns')

In [347]:
# Drop rows where 'playerid' is NaN
lck_data = lck_data.dropna(subset=['playerid'])

# Display the cleaned DataFrame
lck_data

,gameid,league,split,playoffs,date,game,participantid,side,position,playername,playerid,teamname,teamid,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,elders,opp_elders,firstherald,heralds,opp_heralds,void_grubs,opp_void_grubs,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,turretplates,opp_turretplates,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,gpr,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15,goldat20,xpat20,csat20,opp_goldat20,opp_xpat20,opp_csat20,golddiffat20,xpdiffat20,csdiffat20,killsat20,assistsat20,deathsat20,opp_killsat20,opp_assistsat20,opp_deathsat20,goldat25,xpat25,csat25,opp_goldat25,opp_xpat25,opp_csat25,golddiffat25,xpdiffat25,csdiffat25,killsat25,assistsat25,deathsat25,opp_killsat25,opp_assistsat25,opp_deathsat25
0,LOLTMNT02_22165,LCK,Spring,0,2024-01-17 08:20:00,1,1,Blue,top,Rascal,oe:player:9daf04bfd9c8815cc827414f59f45af,DRX,oe:team:101f8589e58c724c1dcd5a9c1555277,2507,0,0,2,4,14,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3351,0.6701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,12881,308.2808,0.119309,879.7048,3157.8620,18,0.4308,13,0.3111,6,61,1.4599,14627,9237,221.0690,0.201840,14350,NaN,NaN,352.0,347.0,5,NaN,NaN,8.4244,3479.0,4733.0,90.0,3237.0,5052.0,86.0,242.0,-319.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,5366.0,7959.0,145.0,4998.0,7764.0,141.0,368.0,195.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,7243.0,10718.0,197.0,6940.0,10604.0,194.0,303.0,114.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,8872.0,12581.0,228.0,9103.0,13341.0,218.0,-231.0,-760.0,10.0,0.0,2.0,1.0,1.0,2.0,3.0
1,LOLTMNT02_22165,LCK,Spring,0,2024-01-17 08:20:00,1,2,Blue,jng,Sponge,oe:player:3f4027dc08a489bd4d5390913f5dc71,DRX,oe:team:101f8589e58c724c1dcd5a9c1555277,2507,0,6,4,3,14,14,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.3351,0.6701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,11256,269.3897,0.104258,889.7806,1548.7036,20,0.4787,20,0.4787,20,79,1.8907,13387,7997,191.3921,0.174745,12500,NaN,NaN,216.0,50.0,166,NaN,NaN,5.1695,3588.0,3651.0,73.0,2874.0,3253.0,58.0,714.0,398.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,5397.0,5531.0,107.0,4519.0,4849.0,76.0,878.0,682.0,31.0,2.0,0.0,0.0,1.0,0.0,1.0,7225.0,7939.0,140.0,6094.0,7462.0,117.0,1131.0,477.0,23.0,3.0,0.0,0.0,1.0,0.0,1.0,8897.0,10105.0,168.0,7934.0,10291.0,147.0,963.0,-186.0,21.0,4.0,0.0,1.0,1.0,2.0,1.0
2,LOLTMNT02_22165,LCK,Spring,0,2024-01-17 08:20:00,1,3,Blue,mid,SeTab,oe:player:4ff7b91bbc9492e9f96d28dda496653,DRX,oe:team:101f8589e58c724c1dcd5a9c1555277,2507,0,2,3,6,14,14,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.3351,0.6701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,25837,618.3566,0.239313,544.7866,452.6446,19,0.4547,10,0.2393,8,43,1.0291,15142,9752,233.3945,0.213093,14000,NaN,NaN,338.0,338.0,0,NaN,NaN,8.0893,3493.0,5088.0,93.0,3015.0,4682.0,75.0,478.0,406.0,18.0,0.0,1.0,0.0,0.0,0.0,1.0,5588.0,8030.0,150.0,4833.0,7571.0,128.0,755.0,459.0,22.0,0.0,1.0,0.0,0.0,0.0,1.0,8101.0,11383.0,206.0,6482.0,10069.0,171.0,1619.0,1314.0,35.0,1.0,1.0,0.0,0.0,1.0,2.0,9566.0,12887.0,221.0,10810.0,14216.0,235.0,-1244.0,-1329.0,-14.0,2.0,3.0,1.0,2.0,1.0,2.0
3,LOLTMNT02_22165,LCK,Spring,0,2024-01-17 08:20:00,1,4,Blue

In [351]:
columns_to_drop = ['game', 'split', 'doublekills', 'triplekills', 'quadrakills', 'pentakills']
existing_columns_to_drop = [col for col in columns_to_drop if col in lck_data.columns]
lck_data.drop(columns=existing_columns_to_drop, inplace=True)
lck_data

/var/folders/0h/ch124vqs7dj_96gjw7qq3ksh0000gn/T/ipykernel_14161/2160997283.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lck_data.drop(columns=existing_columns_to_drop, inplace=True)


,gameid,league,playoffs,date,participantid,side,position,playername,playerid,teamname,teamid,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,elders,opp_elders,firstherald,heralds,opp_heralds,void_grubs,opp_void_grubs,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,turretplates,opp_turretplates,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,gpr,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15,goldat20,xpat20,csat20,opp_goldat20,opp_xpat20,opp_csat20,golddiffat20,xpdiffat20,csdiffat20,killsat20,assistsat20,deathsat20,opp_killsat20,opp_assistsat20,opp_deathsat20,goldat25,xpat25,csat25,opp_goldat25,opp_xpat25,opp_csat25,golddiffat25,xpdiffat25,csdiffat25,killsat25,assistsat25,deathsat25,opp_killsat25,opp_assistsat25,opp_deathsat25
0,LOLTMNT02_22165,LCK,0,2024-01-17 08:20:00,1,Blue,top,Rascal,oe:player:9daf04bfd9c8815cc827414f59f45af,DRX,oe:team:101f8589e58c724c1dcd5a9c1555277,2507,0,0,2,4,14,14,0.0,0.0,0.0,0.0,0.3351,0.6701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,12881,308.2808,0.119309,879.7048,3157.8620,18,0.4308,13,0.3111,6,61,1.4599,14627,9237,221.0690,0.201840,14350,NaN,NaN,352.0,347.0,5,NaN,NaN,8.4244,3479.0,4733.0,90.0,3237.0,5052.0,86.0,242.0,-319.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,5366.0,7959.0,145.0,4998.0,7764.0,141.0,368.0,195.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,7243.0,10718.0,197.0,6940.0,10604.0,194.0,303.0,114.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,8872.0,12581.0,228.0,9103.0,13341.0,218.0,-231.0,-760.0,10.0,0.0,2.0,1.0,1.0,2.0,3.0
1,LOLTMNT02_22165,LCK,0,2024-01-17 08:20:00,2,Blue,jng,Sponge,oe:player:3f4027dc08a489bd4d5390913f5dc71,DRX,oe:team:101f8589e58c724c1dcd5a9c1555277,2507,0,6,4,3,14,14,1.0,1.0,0.0,0.0,0.3351,0.6701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,11256,269.3897,0.104258,889.7806,1548.7036,20,0.4787,20,0.4787,20,79,1.8907,13387,7997,191.3921,0.174745,12500,NaN,NaN,216.0,50.0,166,NaN,NaN,5.1695,3588.0,3651.0,73.0,2874.0,3253.0,58.0,714.0,398.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,5397.0,5531.0,107.0,4519.0,4849.0,76.0,878.0,682.0,31.0,2.0,0.0,0.0,1.0,0.0,1.0,7225.0,7939.0,140.0,6094.0,7462.0,117.0,1131.0,477.0,23.0,3.0,0.0,0.0,1.0,0.0,1.0,8897.0,10105.0,168.0,7934.0,10291.0,147.0,963.0,-186.0,21.0,4.0,0.0,1.0,1.0,2.0,1.0
2,LOLTMNT02_22165,LCK,0,2024-01-17 08:20:00,3,Blue,mid,SeTab,oe:player:4ff7b91bbc9492e9f96d28dda496653,DRX,oe:team:101f8589e58c724c1dcd5a9c1555277,2507,0,2,3,6,14,14,1.0,0.0,1.0,0.0,0.3351,0.6701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,25837,618.3566,0.239313,544.7866,452.6446,19,0.4547,10,0.2393,8,43,1.0291,15142,9752,233.3945,0.213093,14000,NaN,NaN,338.0,338.0,0,NaN,NaN,8.0893,3493.0,5088.0,93.0,3015.0,4682.0,75.0,478.0,406.0,18.0,0.0,1.0,0.0,0.0,0.0,1.0,5588.0,8030.0,150.0,4833.0,7571.0,128.0,755.0,459.0,22.0,0.0,1.0,0.0,0.0,0.0,1.0,8101.0,11383.0,206.0,6482.0,10069.0,171.0,1619.0,1314.0,35.0,1.0,1.0,0.0,0.0,1.0,2.0,9566.0,12887.0,221.0,10810.0,14216.0,235.0,-1244.0,-1329.0,-14.0,2.0,3.0,1.0,2.0,1.0,2.0
3,LOLTMNT02_22165,LCK,0,2024-01-17 08:20:00,4,Blue,bot,Teddy,oe:player:2b5fa564a1259b97bc238fbfa719b07,DRX,oe:team:101f8589e58c724c1dcd5a9c1555277,2507,0,5,4,5,14,14,0.0,0.0,0.0,0.0,0.3351,0.6

In [352]:
# Drop columns where the entire column is NaN
lck_data_cleaned = lck_data.dropna(axis=1, how='all')

# Display the cleaned DataFrame
lck_data_cleaned

,gameid,league,playoffs,date,participantid,side,position,playername,playerid,teamname,teamid,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,barons,opp_barons,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,total cs,minionkills,monsterkills,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15,goldat20,xpat20,csat20,opp_goldat20,opp_xpat20,opp_csat20,golddiffat20,xpdiffat20,csdiffat20,killsat20,assistsat20,deathsat20,opp_killsat20,opp_assistsat20,opp_deathsat20,goldat25,xpat25,csat25,opp_goldat25,opp_xpat25,opp_csat25,golddiffat25,xpdiffat25,csdiffat25,killsat25,assistsat25,deathsat25,opp_killsat25,opp_assistsat25,opp_deathsat25
0,LOLTMNT02_22165,LCK,0,2024-01-17 08:20:00,1,Blue,top,Rascal,oe:player:9daf04bfd9c8815cc827414f59f45af,DRX,oe:team:101f8589e58c724c1dcd5a9c1555277,2507,0,0,2,4,14,14,0.0,0.0,0.0,0.0,0.3351,0.6701,0.0,0.0,0.0,1.0,12881,308.2808,0.119309,879.7048,3157.8620,18,0.4308,13,0.3111,6,61,1.4599,14627,9237,221.0690,0.201840,14350,352.0,347.0,5,8.4244,3479.0,4733.0,90.0,3237.0,5052.0,86.0,242.0,-319.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,5366.0,7959.0,145.0,4998.0,7764.0,141.0,368.0,195.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,7243.0,10718.0,197.0,6940.0,10604.0,194.0,303.0,114.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,8872.0,12581.0,228.0,9103.0,13341.0,218.0,-231.0,-760.0,10.0,0.0,2.0,1.0,1.0,2.0,3.0
1,LOLTMNT02_22165,LCK,0,2024-01-17 08:20:00,2,Blue,jng,Sponge,oe:player:3f4027dc08a489bd4d5390913f5dc71,DRX,oe:team:101f8589e58c724c1dcd5a9c1555277,2507,0,6,4,3,14,14,1.0,1.0,0.0,0.0,0.3351,0.6701,0.0,1.0,0.0,0.0,11256,269.3897,0.104258,889.7806,1548.7036,20,0.4787,20,0.4787,20,79,1.8907,13387,7997,191.3921,0.174745,12500,216.0,50.0,166,5.1695,3588.0,3651.0,73.0,2874.0,3253.0,58.0,714.0,398.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,5397.0,5531.0,107.0,4519.0,4849.0,76.0,878.0,682.0,31.0,2.0,0.0,0.0,1.0,0.0,1.0,7225.0,7939.0,140.0,6094.0,7462.0,117.0,1131.0,477.0,23.0,3.0,0.0,0.0,1.0,0.0,1.0,8897.0,10105.0,168.0,7934.0,10291.0,147.0,963.0,-186.0,21.0,4.0,0.0,1.0,1.0,2.0,1.0
2,LOLTMNT02_22165,LCK,0,2024-01-17 08:20:00,3,Blue,mid,SeTab,oe:player:4ff7b91bbc9492e9f96d28dda496653,DRX,oe:team:101f8589e58c724c1dcd5a9c1555277,2507,0,2,3,6,14,14,1.0,0.0,1.0,0.0,0.3351,0.6701,0.0,0.0,0.0,0.0,25837,618.3566,0.239313,544.7866,452.6446,19,0.4547,10,0.2393,8,43,1.0291,15142,9752,233.3945,0.213093,14000,338.0,338.0,0,8.0893,3493.0,5088.0,93.0,3015.0,4682.0,75.0,478.0,406.0,18.0,0.0,1.0,0.0,0.0,0.0,1.0,5588.0,8030.0,150.0,4833.0,7571.0,128.0,755.0,459.0,22.0,0.0,1.0,0.0,0.0,0.0,1.0,8101.0,11383.0,206.0,6482.0,10069.0,171.0,1619.0,1314.0,35.0,1.0,1.0,0.0,0.0,1.0,2.0,9566.0,12887.0,221.0,10810.0,14216.0,235.0,-1244.0,-1329.0,-14.0,2.0,3.0,1.0,2.0,1.0,2.0
3,LOLTMNT02_22165,LCK,0,2024-01-17 08:20:00,4,Blue,bot,Teddy,oe:player:2b5fa564a1259b97bc238fbfa719b07,DRX,oe:team:101f8589e58c724c1dcd5a9c1555277,2507,0,5,4,5,14,14,0.0,0.0,0.0,0.0,0.3351,0.6701,0.0,2.0,0.0,1.0,46493,1112.7164,0.430638,807.1400,470.2832,18,0.4308,18,0.4308,5,55,1.3163,19772,14382,344.2042,0.314263,20950,392.0,368.0,24,9.3817,3357.0,2859.0,88.0,3285.0,3024.0,88.0,72.0,-165.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6123.0,4849.0,138.0,4699.0,4559.0,132.0,1424.0,290.0,6.0,1.0,0.0,1.0,0.0,0.0,0.0,8373.0,7479.0,193.0,6984.0,6930.0,193.0,1389.0,549.0,0.0,1.0,1.0,2.0,1.0,0.0,0.0,10762.0,9677.0,237.0,9034.0,9949.0,230.0,1728.0,-272.0,7.0,2.0,3.0,2.0,1.0,1.0,1.0
4,LOLTMNT02_22165,LCK,0,2024-01-17 08:20:00,5,Blue,sup,Pleata,oe:player:7fdedd3126915607d5c8f42f8239253,DRX,oe:tea